In [1]:
!pip uninstall -y llama-cpp

### MACOS ONLY, SEE LLAMA.CPP GITHUB FOR OTHER PLATFORMS

In [2]:
!CMAKE_ARGS="-DGGML_METAL=on" pip install llama-cpp-python --upgrade --force-reinstall --no-cache-dir

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸━━━━━━━━━━ 49.5/66.7 MB 41.7 MB/s eta 0:00:01^C
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸━━━━━ 57.7/66.7 MB 42.4 MB/s eta 0:00:01


# download model

In [ ]:
from llama_cpp import Llama

llm = Llama.from_pretrained(
    repo_id="bartowski/Qwen2.5-7B-Instruct-GGUF",
    filename="*Q4_K_M.gguf",
    verbose=False,
    local_dir="models",
)

# 0

In [ ]:
from llama_cpp import Llama

llm = Llama(model_path="models/qwen2-0_5b-instruct-q8_0.gguf", chat_format="chatml")

def extract_knowledge(message):

    llm.create_chat_completion(
        messages=[
            {
                "role": "system",
                "content": "You are a tacit knowledge extractor that only outputs in JSON.",
            },
            {"role": "user", "content": "OpenAI just released an incredible new paper that talks about, look, not diffusion models! That is interesting, because absolutely everyone is talking about diffusion models. These are AI techniques that typically start out from noise, and over time, Reorganize this noise into an image. You see, this concept works for even video generation, it is a series of still images after all, but things get crazier. Diffusion also works in 3D, where the noise shows up as whatever this is. This can be denoised too into virtual character models. Or it works for computer animation too, where the noise can show up as twitching. Insanity. Diffusion models can also perform voice synthesis."},
        ],
        response_format={
            "type": "json_object",
            "schema": {
                "type": "object",
                "properties": {
                    "triplets": {
                        "type": "array",
                        "items": {
                            "type": "object",
                            "properties": {
                                "subject": {"type": "string"},
                                "predicate": {"type": "string"},
                                "object": {"type": "string"}
                            },
                            "required": ["subject", "predicate", "object"]
                        }
                    }
                },
                "required": ["triplets"],
            },
        },
        temperature=0.5,
    )

def speak_to_llama():
    import json

    # Prompt the user for input
    user_message = input("Enter your message: ")

    # Call the extract_knowledge function with the user's message
    response = extract_knowledge(user_message)

    # Parse and print the JSON response
    try:
        response_json = json.loads(response)
        print("Extracted Knowledge Triplets:")
        for triplet in response_json.get("triplets", []):
            print(f"Subject: {triplet['subject']}, Predicate: {triplet['predicate']}, Object: {triplet['object']}")
    except json.JSONDecodeError:
        print("Failed to parse the response as JSON.")


In [ ]:
speak_to_llama()

In [ ]:
import json
from llama_cpp import Llama
from sqlalchemy import create_engine, Column, Integer, String, Text, ForeignKey
from sqlalchemy.orm import sessionmaker, declarative_base, relationship
import os

# Database setup
Base = declarative_base()

class Message(Base):
    __tablename__ = 'messages'
    id = Column(Integer, primary_key=True)
    role = Column(String(10))  # 'user' or 'assistant'
    content = Column(Text)
    triplets = relationship("Triplet", back_populates="message")

class Triplet(Base):
    __tablename__ = 'triplets'
    id = Column(Integer, primary_key=True)
    subject = Column(String(255))
    predicate = Column(String(255))
    object = Column(String(255))
    message_id = Column(Integer, ForeignKey('messages.id'))
    message = relationship("Message", back_populates="triplets")

# Initialize database
engine = create_engine('sqlite:///chatbot.db')
Base.metadata.create_all(engine)
Session = sessionmaker(bind=engine)
session = Session()

# Initialize LLaMA model
llm = Llama(model_path="models/qwen2-0_5b-instruct-q8_0.gguf", chat_format="chatml")

def extract_knowledge(message):
    response = llm.create_chat_completion(
        messages=[
            {
                "role": "system",
                "content": "You are a tacit knowledge extractor that only outputs in JSON.",
            },
            {"role": "user", "content": message},
        ],
        response_format={
            "type": "json_object",
            "schema": {
                "type": "object",
                "properties": {
                    "triplets": {
                        "type": "array",
                        "items": {
                            "type": "object",
                            "properties": {
                                "subject": {"type": "string"},
                                "predicate": {"type": "string"},
                                "object": {"type": "string"}
                            },
                            "required": ["subject", "predicate", "object"]
                        }
                    }
                },
                "required": ["triplets"],
            },
        },
        temperature=0.5,
    )

    #save json response
    save_message(role='assistant', content=json.dumps(response))
    return json.loads(response['choices'][0]['message']['content'])

def save_message(role, content, extracted_triplets=None):
    msg = Message(role=role, content=content)
    session.add(msg)
    session.commit()
    if extracted_triplets:
        for triplet in extracted_triplets:
            trip = Triplet(
                subject=triplet.get('subject', ''),
                predicate=triplet.get('predicate', ''),
                object=triplet.get('object', ''),
                message_id=msg.id
            )
            session.add(trip)
        session.commit()

def get_recent_conversation(limit=5):
    messages = session.query(Message).order_by(Message.id.desc()).limit(limit).all()
    # Reverse to maintain chronological order
    messages = messages[::-1]
    conversation = []
    for msg in messages:
        conversation.append({"role": msg.role, "content": msg.content})
    return conversation

def generate_response(conversation_history):
    response = llm.create_chat_completion(
        messages=conversation_history,
        temperature=0.7,
    )
    return response['choices'][0]['message']['content']

def speak_to_llama():
    print("Chatbot is ready! Type 'exit' to end the conversation.")
    while True:
        user_message = input("You: ")
        if user_message.lower() in ['exit', 'quit']:
            print("Chatbot: Goodbye!")
            break

        # Save user message
        save_message(role='user', content=user_message)

        # Extract knowledge from user message
        user_triplet_response = extract_knowledge(user_message)
        user_triplets = user_triplet_response.get("triplets", [])
        save_message(role='user', content=user_message, extracted_triplets=user_triplets)

        # Retrieve recent conversation for context
        conversation = get_recent_conversation(limit=10)  # Adjust limit as needed

        # Generate assistant response
        assistant_response = generate_response(conversation)
        print(f"Assistant: {assistant_response}")

        # Save assistant response
        save_message(role='assistant', content=assistant_response)

        # Extract knowledge from assistant response
        assistant_triplet_response = extract_knowledge(assistant_response)
        assistant_triplets = assistant_triplet_response.get("triplets", [])
        save_message(role='assistant', content=assistant_response, extracted_triplets=assistant_triplets)

if __name__ == "__main__":
    speak_to_llama()

In [ ]:
import json
from llama_cpp import Llama
from sqlalchemy import create_engine, Column, Integer, String, Text, ForeignKey
from sqlalchemy.orm import sessionmaker, declarative_base, relationship
import os
import threading

# Database setup
Base = declarative_base()

class Message(Base):
    __tablename__ = 'messages'
    id = Column(Integer, primary_key=True)
    role = Column(String(10))  # 'user' or 'assistant'
    content = Column(Text)
    triplets = relationship("Triplet", back_populates="message")

class Triplet(Base):
    __tablename__ = 'triplets'
    id = Column(Integer, primary_key=True)
    subject = Column(String(255))
    predicate = Column(String(255))
    object = Column(String(255))
    message_id = Column(Integer, ForeignKey('messages.id'))
    message = relationship("Message", back_populates="triplets")

# Initialize database
engine = create_engine('sqlite:///chatbot.db')
Base.metadata.create_all(engine)
Session = sessionmaker(bind=engine)
session = Session()

# Singleton pattern for LLaMA model
class LlamaSingleton:
    _instance = None
    _lock = threading.Lock()

    def __new__(cls, model_path="models/qwen2-0_5b-instruct-q8_0.gguf", chat_format="chatml"):
        with cls._lock:
            if cls._instance is None:
                cls._instance = super(LlamaSingleton, cls).__new__(cls)
                cls._instance.llm = Llama(model_path=model_path, chat_format=chat_format)
            return cls._instance

def extract_valuable_knowledge(message):
    llama_instance = LlamaSingleton().llm
    response = llama_instance.create_chat_completion(
        messages=[
            {
                "role": "system",
                "content": "You are a knowledge extractor that identifies and outputs only valuable information in JSON format.",
            },
            {"role": "user", "content": message},
        ],
        response_format={
            "type": "json_object",
            "schema": {
                "type": "object",
                "properties": {
                    "valuable_knowledge": {
                        "type": "array",
                        "items": {
                            "type": "object",
                            "properties": {
                                "subject": {"type": "string"},
                                "predicate": {"type": "string"},
                                "object": {"type": "string"}
                            },
                            "required": ["subject", "predicate", "object"]
                        }
                    }
                },
                "required": ["valuable_knowledge"],
            },
        },
        temperature=0.5,
    )
    return json.loads(response['choices'][0]['message']['content'])

def save_knowledge_to_json(knowledge, filename='valuable_knowledge.json'):
    try:
        with open(filename, 'r') as file:
            existing_knowledge = json.load(file)
    except (FileNotFoundError, json.JSONDecodeError):
        existing_knowledge = []

    existing_knowledge.extend(knowledge)

    with open(filename, 'w') as file:
        json.dump(existing_knowledge, file, indent=4)

def load_knowledge_from_json(filename='valuable_knowledge.json'):
    try:
        with open(filename, 'r') as file:
            return json.load(file)
    except (FileNotFoundError, json.JSONDecodeError):
        return []

def save_message(role, content, extracted_knowledge=None):
    msg = Message(role=role, content=content)
    session.add(msg)
    session.commit()
    if extracted_knowledge:
        for knowledge in extracted_knowledge:
            trip = Triplet(
                subject=knowledge.get('subject', ''),
                predicate=knowledge.get('predicate', ''),
                object=knowledge.get('object', ''),
                message_id=msg.id
            )
            session.add(trip)
        session.commit()
        save_knowledge_to_json(extracted_knowledge)

def get_recent_conversation(limit=5):
    messages = session.query(Message).order_by(Message.id.desc()).limit(limit).all()
    messages = messages[::-1]
    conversation = []
    for msg in messages:
        conversation.append({"role": msg.role, "content": msg.content})
    return conversation

def generate_response(conversation_history):
    llama_instance = LlamaSingleton().llm
    knowledge_triplets = load_knowledge_from_json()
    knowledge_context = "\n".join(
        f"{triplet['subject']} {triplet['predicate']} {triplet['object']}"
        for triplet in knowledge_triplets
    )
    conversation_history.insert(0, {"role": "system", "content": knowledge_context})
    response = llama_instance.create_chat_completion(
        messages=conversation_history,
        temperature=0.7,
    )
    return response['choices'][0]['message']['content']

def speak_to_llama():
    print("Chatbot is ready! Type 'exit' to end the conversation.")
    while True:
        user_message = input("You: ")
        if user_message.lower() in ['exit', 'quit']:
            print("Chatbot: Goodbye!")
            break

        save_message(role='user', content=user_message)

        user_knowledge_response = extract_valuable_knowledge(user_message)
        user_knowledge = user_knowledge_response.get("valuable_knowledge", [])
        save_message(role='user', content=user_message, extracted_knowledge=user_knowledge)

        conversation = get_recent_conversation(limit=10)

        assistant_response = generate_response(conversation)
        print(f"Assistant: {assistant_response}")

        save_message(role='assistant', content=assistant_response)

        assistant_knowledge_response = extract_valuable_knowledge(assistant_response)
        assistant_knowledge = assistant_knowledge_response.get("valuable_knowledge", [])
        save_message(role='assistant', content=assistant_response, extracted_knowledge=assistant_knowledge)

if __name__ == "__main__":
    speak_to_llama()

In [ ]:
import json
import os
from datetime import datetime
from llama_cpp import Llama
import threading

# Singleton pattern for LLaMA model
class LlamaSingleton:
    _instance = None
    _lock = threading.Lock()

    def __new__(cls, model_path="models/qwen2-0_5b-instruct-q8_0.gguf", chat_format="chatml"):
        with cls._lock:
            if cls._instance is None:
                cls._instance = super(LlamaSingleton, cls).__new__(cls)
                cls._instance.llm = Llama(model_path=model_path, chat_format=chat_format)
            return cls._instance

# File paths
MESSAGES_FILE = 'messages.json'
KNOWLEDGE_FILE = 'knowledge.json'

# Initialize JSON files if they don't exist
def initialize_json_files():
    for file in [MESSAGES_FILE, KNOWLEDGE_FILE]:
        if not os.path.exists(file):
            with open(file, 'w') as f:
                json.dump([], f)

# Load JSON data from a file
def load_json_data(file_path):
    with open(file_path, 'r') as f:
        return json.load(f)

# Save JSON data to a file
def save_json_data(file_path, data):
    with open(file_path, 'w') as f:
        json.dump(data, f, indent=4)

# Extract valuable knowledge from a message
def extract_valuable_knowledge(message):
    llama_instance = LlamaSingleton().llm
    response = llama_instance.create_chat_completion(
        messages=[
            {
                "role": "system",
                "content": "You are a knowledge extractor that identifies and outputs only valuable information in JSON format.",
            },
            {"role": "user", "content": message},
        ],
        response_format={
            "type": "json",
            "schema": {
                "type": "object",
                "properties": {
                    "valuable_knowledge": {
                        "type": "array",
                        "items": {
                            "type": "object",
                            "properties": {
                                "subject": {"type": "string"},
                                "predicate": {"type": "string"},
                                "object": {"type": "string"}
                            },
                            "required": ["subject", "predicate", "object"]
                        }
                    }
                },
                "required": ["valuable_knowledge"],
            },
        },
        temperature=0.5,
    )
    return json.loads(response['choices'][0]['message']['content'])

# Save a message to messages.json
def save_message(role, content):
    messages = load_json_data(MESSAGES_FILE)
    message = {
        "role": role,
        "content": content,
        "timestamp": datetime.utcnow().isoformat()
    }
    messages.append(message)
    save_json_data(MESSAGES_FILE, messages)

# Save extracted knowledge to knowledge.json
def save_knowledge(triplets):
    knowledge = load_json_data(KNOWLEDGE_FILE)
    for triplet in triplets:
        triplet['timestamp'] = datetime.utcnow().isoformat()
        knowledge.append(triplet)
    save_json_data(KNOWLEDGE_FILE, knowledge)

# Get recent conversation history
def get_recent_conversation(limit=5):
    messages = load_json_data(MESSAGES_FILE)
    return messages[-limit:]

# Search knowledge triplets for relevant information
def search_knowledge_triplets(query):
    knowledge_triplets = load_json_data(KNOWLEDGE_FILE)
    relevant_triplets = []
    for triplet in knowledge_triplets:
        if query.lower() in (triplet['subject'].lower(), triplet['predicate'].lower(), triplet['object'].lower()):
            relevant_triplets.append(triplet)
    return relevant_triplets

# Generate a response based on conversation history and knowledge
def generate_response(conversation_history, user_message):
    llama_instance = LlamaSingleton().llm
    knowledge_triplets = search_knowledge_triplets(user_message)
    if knowledge_triplets:
        knowledge_context = "\n".join(
            f"{triplet['subject']} {triplet['predicate']} {triplet['object']} (Timestamp: {triplet['timestamp']})"
            for triplet in knowledge_triplets
        )
        response = f"Based on what I know:\n{knowledge_context}"
    else:
        conversation_history.append({"role": "user", "content": user_message})
        response = llama_instance.create_chat_completion(
            messages=conversation_history,
            temperature=0.7,
        )['choices'][0]['message']['content']
    return response

# Main chat function
def speak_to_llama():
    initialize_json_files()
    print("Chatbot is ready! Type 'exit' to end the conversation.")
    while True:
        user_message = input("You: ")
        if user_message.lower() in ['exit', 'quit']:
            print("Chatbot: Goodbye!")
            break

        # Save user message
        save_message(role='user', content=user_message)

        # Retrieve recent conversation for context
        conversation = get_recent_conversation(limit=10)

        # Generate assistant response
        assistant_response = generate_response(conversation, user_message)
        print(f"Assistant: {assistant_response}")

        # Save assistant response
        save_message(role='assistant', content=assistant_response)

        # Extract and save knowledge from user message
        user_knowledge_response = extract_valuable_knowledge(user_message)
        user_knowledge = user_knowledge_response.get("valuable_knowledge", [])
        save_knowledge(user_knowledge)

if __name__ == "__main__":
    speak_to_llama()

In [ ]:
import json
import os
from datetime import datetime
from llama_cpp import Llama
import threading

class LlamaSingleton:
    _instance = None
    _lock = threading.Lock()

    def __new__(cls, model_path="models/qwen2-0_5b-instruct-q8_0.gguf", chat_format="chatml"):
        with cls._lock:
            if cls._instance is None:
                cls._instance = super(LlamaSingleton, cls).__new__(cls)
                cls._instance.llm = Llama(model_path=model_path, chat_format=chat_format, n_ctx=1024)
            return cls._instance

class Chatbot:
    def __init__(self, messages_file='messages.json', knowledge_file='knowledge.json'):
        self.messages_file = messages_file
        self.knowledge_file = knowledge_file
        self.llm = LlamaSingleton().llm
        self.initialize_json_files()

    def initialize_json_files(self):
        for file in [self.messages_file, self.knowledge_file]:
            if not os.path.exists(file):
                with open(file, 'w') as f:
                    json.dump([], f)

    def load_json_data(self, file_path):
        with open(file_path, 'r') as f:
            return json.load(f)

    def save_json_data(self, file_path, data):
        with open(file_path, 'w') as f:
            json.dump(data, f, indent=4)

    def extract_valuable_knowledge(self, message):
        response = self.llm.create_chat_completion(
            messages=[
                {
                    "role": "system",
                    "content": "You are a knowledge extractor that identifies and outputs only valuable information in JSON format.",
                },
                {"role": "user", "content": message},
            ],
            response_format={
                "type": "json",
                "schema": {
                    "type": "object",
                    "properties": {
                        "valuable_knowledge": {
                            "type": "array",
                            "items": {
                                "type": "object",
                                "properties": {
                                    "subject": {"type": "string"},
                                    "predicate": {"type": "string"},
                                    "object": {"type": "string"},
                                    "timestamp": {"type": "string", "format": "date-time"}
                                },
                                "required": ["subject", "predicate", "object", "timestamp"]
                            }
                        }
                    },
                    "required": ["valuable_knowledge"],
                },
            },
            temperature=0.5,
        )
        return json.loads(response['choices'][0]['message']['content'])

    def save_message(self, role, content):
        messages = self.load_json_data(self.messages_file)
        message = {
            "role": role,
            "content": content,
            "timestamp": datetime.utcnow().isoformat()
        }
        messages.append(message)
        self.save_json_data(self.messages_file, messages)

    def save_knowledge(self, triplets):
        knowledge = self.load_json_data(self.knowledge_file)
        for triplet in triplets:
            triplet['timestamp'] = datetime.utcnow().isoformat()
            knowledge.append(triplet)
        self.save_json_data(self.knowledge_file, knowledge)

    def get_recent_conversation(self, limit=5):
        messages = self.load_json_data(self.messages_file)
        return messages[-limit:]

    def search_knowledge_triplets(self, query):
        knowledge_triplets = self.load_json_data(self.knowledge_file)
        relevant_triplets = []
        for triplet in knowledge_triplets:
            if query.lower() in (triplet['subject'].lower(), triplet['predicate'].lower(), triplet['object'].lower()):
                relevant_triplets.append(triplet)
        return relevant_triplets

    def generate_response(self, conversation_history, user_message):
        knowledge_triplets = self.search_knowledge_triplets(user_message)
        current_time = datetime.utcnow().isoformat()
        system_message = f"Current date and time: {current_time}\n"
        if knowledge_triplets:
            knowledge_context = "\n".join(
                f"{triplet['subject']} {triplet['predicate']} {triplet['object']} (Timestamp: {triplet['timestamp']})"
                for triplet in knowledge_triplets
            )
            system_message += f"Based on existing knowledge:\n{knowledge_context}\n"
        conversation_history.insert(0, {"role": "system", "content": system_message})
        conversation_history.append({"role": "user", "content": user_message})
        response = self.llm.create_chat_completion(
            messages=conversation_history,
            temperature=0.7,
        )['choices'][0]['message']['content']
        return response

    def chat(self):
        print("Chatbot is ready! Type 'exit' to end the conversation.")
        while True:
            user_message = input("You: ")
            if user_message.lower() in ['exit', 'quit']:
                print("Chatbot: Goodbye!")
                break

            self.save_message(role='user', content=user_message)

            conversation = self.get_recent_conversation(limit=10)

            assistant_response = self.generate_response(conversation, user_message)
            print(f"Assistant: {assistant_response}")

            self.save_message(role='assistant', content=assistant_response)

            user_knowledge_response = self.extract_valuable_knowledge(user_message)
            user_knowledge = user_knowledge_response.get("valuable_knowledge", [])
            self.save_knowledge(user_knowledge)

if __name__ == "__main__":
    chatbot = Chatbot()
    chatbot.chat()

In [ ]:
import json
import os
import pickle
from datetime import datetime
from llama_cpp import Llama
import threading
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

class LlamaSingleton:
    _instance = None
    _lock = threading.Lock()

    def __new__(cls, model_path="models/qwen2-0_5b-instruct-q8_0.gguf", chat_format="chatml"):
        with cls._lock:
            if cls._instance is None:
                cls._instance = super(LlamaSingleton, cls).__new__(cls)
                cls._instance.llm = Llama(model_path=model_path, chat_format=chat_format, n_ctx=1024)
            return cls._instance

class Chatbot:
    def __init__(self, messages_file='messages.json', knowledge_file='knowledge.json', faiss_index_file='faiss_index.pkl'):
        self.messages_file = messages_file
        self.knowledge_file = knowledge_file
        self.faiss_index_file = faiss_index_file
        self.llm = LlamaSingleton().llm
        self.model = SentenceTransformer('all-MiniLM-L6-v2')
        self.index = None
        self.knowledge_data = []
        self.initialize_files()
        self.load_faiss_index()

    def initialize_files(self):
        for file in [self.messages_file, self.knowledge_file]:
            if not os.path.exists(file):
                with open(file, 'w') as f:
                    json.dump([], f)

    def load_json_data(self, file_path):
        with open(file_path, 'r') as f:
            return json.load(f)

    def save_json_data(self, file_path, data):
        with open(file_path, 'w') as f:
            json.dump(data, f, indent=4)

    def extract_valuable_knowledge(self, message):
        response = self.llm.create_chat_completion(
            messages=[
                {
                    "role": "system",
                    "content": "You are a knowledge extractor that identifies and outputs only valuable information in JSON format.",
                },
                {"role": "user", "content": message},
            ],
            response_format={
                "type": "json",
                "schema": {
                    "type": "object",
                    "properties": {
                        "valuable_knowledge": {
                            "type": "array",
                            "items": {
                                "type": "object",
                                "properties": {
                                    "subject": {"type": "string"},
                                    "predicate": {"type": "string"},
                                    "object": {"type": "string"},
                                    "timestamp": {"type": "string", "format": "date-time"}
                                },
                                "required": ["subject", "predicate", "object", "timestamp"]
                            }
                        }
                    },
                    "required": ["valuable_knowledge"],
                },
            },
            temperature=0.5,
        )
        return json.loads(response['choices'][0]['message']['content'])

    def save_message(self, role, content):
        messages = self.load_json_data(self.messages_file)
        message = {"role": role, "content": content, "timestamp": datetime.utcnow().isoformat()}
        messages.append(message)
        self.save_json_data(self.messages_file, messages)

    def save_knowledge(self, triplets):
        knowledge = self.load_json_data(self.knowledge_file)
        new_triplets = []
        for triplet in triplets:
            triplet['timestamp'] = datetime.utcnow().isoformat()
            knowledge.append(triplet)
            new_triplets.append(triplet)
        self.save_json_data(self.knowledge_file, knowledge)
        self.update_faiss_index(new_triplets)

    def update_faiss_index(self, triplets):
        texts = [f"{t['subject']} {t['predicate']} {t['object']}" for t in triplets]
        embeddings = self.model.encode(texts)
        if self.index is None:
            self.index = faiss.IndexFlatL2(embeddings.shape[1])
        self.index.add(np.array(embeddings, dtype=np.float32))
        self.knowledge_data.extend(triplets)
        self.save_faiss_index()

    def save_faiss_index(self):
        with open(self.faiss_index_file, 'wb') as f:
            pickle.dump((self.index, self.knowledge_data), f)

    def load_faiss_index(self):
        if os.path.exists(self.faiss_index_file):
            with open(self.faiss_index_file, 'rb') as f:
                self.index, self.knowledge_data = pickle.load(f)
        else:
            self.index = None
            self.knowledge_data = []

    def search_knowledge(self, query, top_k=5):
        if self.index is None:
            return []
        query_embedding = self.model.encode([query])
        distances, indices = self.index.search(np.array(query_embedding, dtype=np.float32), top_k)
        results = [self.knowledge_data[idx] for idx in indices[0] if idx != -1]
        return results

    def generate_response(self, conversation_history, user_message):
        knowledge_matches = self.search_knowledge(user_message)
        current_time = datetime.utcnow().isoformat()
        system_message = f"Current date and time: {current_time}\n"
        if knowledge_matches:
            knowledge_context = "\n".join(
                f"{t['subject']} {t['predicate']} {t['object']} (Added on: {t['timestamp']})"
                for t in knowledge_matches
            )
            system_message += f"Based on my knowledge:\n{knowledge_context}\n"
        conversation_history.insert(0, {"role": "system", "content": system_message})
        conversation_history.append({"role": "user", "content": user_message})
        response = self.llm.create_chat_completion(
            messages=conversation_history,
            temperature=0.7,
        )['choices'][0]['message']['content']
        return response

    def chat(self):
        print("Chatbot is ready! Type 'exit' to end the conversation.")
        while True:
            user_message = input("You: ")
            if user_message.lower() in ['exit', 'quit']:
                print("Chatbot: Goodbye!")
                break

            self.save_message(role='user', content=user_message)

            conversation = self.load_json_data(self.messages_file)[-10:]

            assistant_response = self.generate_response(conversation, user_message)
            print(f"Assistant: {assistant_response}")

            self.save_message(role='assistant', content=assistant_response)

            user_knowledge_response = self.extract_valuable_knowledge(user_message)
            if user_knowledge_response:
                self.save_knowledge(user_knowledge_response)
            else:
                print("No valuable knowledge extracted; continuing conversation.")

if __name__ == "__main__":
    chatbot = Chatbot()
    chatbot.chat()

In [ ]:
import json
import os
import pickle
from datetime import datetime
from llama_cpp import Llama
import threading
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from json import JSONDecodeError

class LlamaSingleton:
    """A thread-safe singleton for loading a single Llama model instance."""
    _instance = None
    _lock = threading.Lock()

    def __new__(cls, model_path="models/qwen2-0_5b-instruct-q8_0.gguf", chat_format="chatml"):
        with cls._lock:
            if cls._instance is None:
                cls._instance = super(LlamaSingleton, cls).__new__(cls)
                cls._instance.llm = Llama(model_path=model_path, chat_format=chat_format, n_ctx=1024)
            return cls._instance


class Chatbot:
    def __init__(self, 
                 messages_file='messages.json', 
                 knowledge_file='knowledge.json', 
                 faiss_index_file='faiss_index.pkl',
                 model_name='all-MiniLM-L6-v2'):
        """
        Initialize the chatbot, ensuring that the files exist and loading FAISS index if available.
        """
        self.messages_file = messages_file
        self.knowledge_file = knowledge_file
        self.faiss_index_file = faiss_index_file

        # Initialize LLM and embedding model
        self.llm = LlamaSingleton().llm
        self.model = SentenceTransformer(model_name)

        # Internal memory of knowledge and FAISS index
        self.index = None
        self.knowledge_data = []

        # Ensure JSON files exist
        self.initialize_files()

        # Load any existing FAISS index
        self.load_faiss_index()

    def initialize_files(self):
        """Ensure that the message and knowledge JSON files exist."""
        for file in [self.messages_file, self.knowledge_file]:
            if not os.path.exists(file):
                with open(file, 'w') as f:
                    json.dump([], f)

    def load_json_data(self, file_path):
        """Load data from a JSON file."""
        with open(file_path, 'r') as f:
            return json.load(f)

    def save_json_data(self, file_path, data):
        """Save data to a JSON file."""
        with open(file_path, 'w') as f:
            json.dump(data, f, indent=4)

    def extract_valuable_knowledge(self, message):
        """
        Extract valuable knowledge from a user message using the LLM.
        The LLM is prompted to return JSON structured knowledge.
        """
        # Prompt the model to return valuable knowledge triplets in JSON format.
        response = self.llm.create_chat_completion(
            messages=[
                {
                    "role": "system",
                    "content": (
                        "You are a knowledge extractor. Extract valuable knowledge from the user's message.\n"
                        "Return ONLY JSON with the following schema:\n\n"
                        "{\n"
                        "  \"valuable_knowledge\": [\n"
                        "    {\n"
                        "      \"subject\": \"...\",\n"
                        "      \"predicate\": \"...\",\n"
                        "      \"object\": \"...\",\n"
                        "      \"timestamp\": \"...\"  # ISO8601\n"
                        "    }\n"
                        "  ]\n"
                        "}\n\n"
                        "If no knowledge can be extracted, return:\n"
                        "{\"valuable_knowledge\": []}"
                    )
                },
                {"role": "user", "content": message},
            ],
            response_format={
                "type": "json",
                "schema": {
                    "type": "object",
                    "properties": {
                        "valuable_knowledge": {
                            "type": "array",
                            "items": {
                                "type": "object",
                                "properties": {
                                    "subject": {"type": "string"},
                                    "predicate": {"type": "string"},
                                    "object": {"type": "string"},
                                    "timestamp": {"type": "string", "format": "date-time"}
                                },
                                "required": ["subject", "predicate", "object", "timestamp"]
                            }
                        }
                    },
                    "required": ["valuable_knowledge"],
                },
            },
            temperature=0.5,
        )

        # Try parsing the returned JSON.
        try:
            knowledge_data = json.loads(response['choices'][0]['message']['content'])
            # Ensure the result is at least an empty list if no knowledge was found.
            if "valuable_knowledge" not in knowledge_data:
                knowledge_data["valuable_knowledge"] = []
            return knowledge_data["valuable_knowledge"]
        except (JSONDecodeError, KeyError):
            # If there's any parsing error, return empty knowledge.
            return []

    def save_message(self, role, content):
        """Append a new message to the messages file."""
        messages = self.load_json_data(self.messages_file)
        message = {"role": role, "content": content, "timestamp": datetime.utcnow().isoformat()}
        messages.append(message)
        self.save_json_data(self.messages_file, messages)

    def save_knowledge(self, triplets):
        """
        Save extracted knowledge triplets to the knowledge file and update the FAISS index.
        De-duplicates knowledge based on (subject, predicate, object).
        """
        if not triplets:
            return

        knowledge = self.load_json_data(self.knowledge_file)

        # Create a set of existing triples to avoid duplicates
        existing_set = {(t['subject'], t['predicate'], t['object']) for t in knowledge}

        new_triplets = []
        for triplet in triplets:
            # Ensure timestamp is properly updated
            triplet['timestamp'] = datetime.utcnow().isoformat()
            key = (triplet['subject'], triplet['predicate'], triplet['object'])
            if key not in existing_set:
                knowledge.append(triplet)
                new_triplets.append(triplet)
                existing_set.add(key)

        # Update knowledge file
        self.save_json_data(self.knowledge_file, knowledge)

        # Update FAISS index if we have new triplets
        if new_triplets:
            self.update_faiss_index(new_triplets)

    def update_faiss_index(self, triplets):
        """
        Update the FAISS index with new triplets.
        Each triplet is embedded and added to the index.
        """
        texts = [f"{t['subject']} {t['predicate']} {t['object']}" for t in triplets]
        embeddings = self.model.encode(texts)

        if self.index is None:
            self.index = faiss.IndexFlatL2(embeddings.shape[1])

        self.index.add(np.array(embeddings, dtype=np.float32))
        self.knowledge_data.extend(triplets)
        self.save_faiss_index()

    def save_faiss_index(self):
        """Save the FAISS index and knowledge data to a file."""
        with open(self.faiss_index_file, 'wb') as f:
            pickle.dump((self.index, self.knowledge_data), f)

    def load_faiss_index(self):
        """Load the FAISS index and knowledge data if available."""
        if os.path.exists(self.faiss_index_file):
            with open(self.faiss_index_file, 'rb') as f:
                self.index, self.knowledge_data = pickle.load(f)
        else:
            self.index = None
            self.knowledge_data = []

    def search_knowledge(self, query, top_k=5):
        """
        Search the FAISS index for the top_k most relevant knowledge triplets related to the query.

        :param query: The query string to search for.
        :param top_k: The number of top results to return.
        :return: A list of triplets (each a dict with 'subject', 'predicate', 'object', 'timestamp').
        """
        # If no index or knowledge is loaded, return an empty list
        if self.index is None or len(self.knowledge_data) == 0:
            return []

        # Encode the query to its embedding
        query_embedding = self.model.encode([query])

        # Perform the search on the FAISS index
        distances, indices = self.index.search(np.array(query_embedding, dtype=np.float32), top_k)

        results = []
        for idx in indices[0]:
            # If FAISS didn't return a valid index (-1), skip it
            if idx == -1:
                continue
            results.append(self.knowledge_data[idx])
        return results

    def generate_response(self, conversation_history, user_message):
        """
        Generate a response using the LLM, enriched with knowledge context if available.
        """
        knowledge_matches = self.search_knowledge(user_message)
        current_time = datetime.utcnow().isoformat()
        
        # Build a system message with current time and matched knowledge
        system_message = f"Current date and time: {current_time}\n"
        if knowledge_matches:
            system_message += "Based on retrieved knowledge:\n"
            for t in knowledge_matches:
                system_message += f"- {t['subject']} {t['predicate']} {t['object']} (Added on: {t['timestamp']})\n"
        else:
            system_message += "No direct related knowledge found. Proceeding with general reasoning.\n"

        # Prepend system message and append user message
        enriched_history = [{"role": "system", "content": system_message}] + conversation_history
        enriched_history.append({"role": "user", "content": user_message})

        response = self.llm.create_chat_completion(
            messages=enriched_history,
            temperature=0.7,
        )['choices'][0]['message']['content']
        return response

    def chat(self):
        """
        Start the interactive chat loop.
        """
        print("Chatbot is ready! Type 'exit' to end the conversation.")
        while True:
            user_message = input("You: ")
            if user_message.lower().strip() in ['exit', 'quit']:
                print("Chatbot: Goodbye!")
                break

            # Save user message
            self.save_message(role='user', content=user_message)

            # Load recent conversation (last 10 messages to maintain some context)
            conversation = self.load_json_data(self.messages_file)[-10:]
            # Generate assistant response
            assistant_response = self.generate_response(conversation, user_message)
            print(f"Assistant: {assistant_response}")

            # Save assistant response
            self.save_message(role='assistant', content=assistant_response)

            # Extract and save new knowledge from the user's last message
            user_knowledge_response = self.extract_valuable_knowledge(user_message)
            if user_knowledge_response:
                self.save_knowledge(user_knowledge_response)
            else:
                # If no knowledge was extracted, we continue silently
                pass


if __name__ == "__main__":
    chatbot = Chatbot()
    query = "What do we know about topic X?"
    search_results = chatbot.search_knowledge(query, top_k=3)
    if search_results:
        print("Top 3 relevant knowledge triplets:")
        for r in search_results:
            print(f"- {r['subject']} {r['predicate']} {r['object']} (timestamp: {r['timestamp']})")
    else:
        print("No relevant knowledge found for that query.")

In [ ]:
import json
import os
import pickle
from datetime import datetime
from llama_cpp import Llama
import threading
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from json import JSONDecodeError

class LlamaSingleton:
    _instance = None
    _lock = threading.Lock()

    def __new__(cls, model_path="models/qwen2-0_5b-instruct-q8_0.gguf", chat_format="chatml"):
        with cls._lock:
            if cls._instance is None:
                cls._instance = super(LlamaSingleton, cls).__new__(cls)
                cls._instance.llm = Llama(model_path=model_path, chat_format=chat_format, n_ctx=1024)
            return cls._instance


class Chatbot:
    def __init__(self, 
                 messages_file='messages.json', 
                 knowledge_file='knowledge.json', 
                 faiss_index_file='faiss_index.pkl',
                 model_name='all-MiniLM-L6-v2'):
        self.messages_file = messages_file
        self.knowledge_file = knowledge_file
        self.faiss_index_file = faiss_index_file
        self.llm = LlamaSingleton().llm
        self.model = SentenceTransformer(model_name)
        self.index = None
        self.knowledge_data = []
        self.initialize_files()
        self.load_faiss_index()

    def initialize_files(self):
        for file in [self.messages_file, self.knowledge_file]:
            if not os.path.exists(file):
                with open(file, 'w') as f:
                    json.dump([], f)

    def load_json_data(self, file_path):
        with open(file_path, 'r') as f:
            return json.load(f)

    def save_json_data(self, file_path, data):
        with open(file_path, 'w') as f:
            json.dump(data, f, indent=4)

    def extract_valuable_knowledge(self, message):
        response = self.llm.create_chat_completion(
            messages=[
                {
                    "role": "system",
                    "content": (
                        "You are a knowledge extractor. Extract valuable knowledge from the user's message.\n"
                        "Return ONLY JSON with the following schema:\n\n"
                        "{\n"
                        "  \"valuable_knowledge\": [\n"
                        "    {\n"
                        "      \"subject\": \"...\",\n"
                        "      \"predicate\": \"...\",\n"
                        "      \"object\": \"...\",\n"
                        "      \"timestamp\": \"...\"  # ISO8601\n"
                        "    }\n"
                        "  ]\n"
                        "}\n\n"
                        "If no knowledge can be extracted, return:\n"
                        "{\"valuable_knowledge\": []}"
                    )
                },
                {"role": "user", "content": message},
            ],
            response_format={
                "type": "json",
                "schema": {
                    "type": "object",
                    "properties": {
                        "valuable_knowledge": {
                            "type": "array",
                            "items": {
                                "type": "object",
                                "properties": {
                                    "subject": {"type": "string"},
                                    "predicate": {"type": "string"},
                                    "object": {"type": "string"},
                                    "timestamp": {"type": "string", "format": "date-time"}
                                },
                                "required": ["subject", "predicate", "object", "timestamp"]
                            }
                        }
                    },
                    "required": ["valuable_knowledge"],
                },
            },
            temperature=0.5,
        )
        try:
            knowledge_data = json.loads(response['choices'][0]['message']['content'])
            if "valuable_knowledge" not in knowledge_data:
                knowledge_data["valuable_knowledge"] = []
            return knowledge_data["valuable_knowledge"]
        except (JSONDecodeError, KeyError):
            return []

    def save_message(self, role, content):
        messages = self.load_json_data(self.messages_file)
        message = {"role": role, "content": content, "timestamp": datetime.utcnow().isoformat()}
        messages.append(message)
        self.save_json_data(self.messages_file, messages)

    def save_knowledge(self, triplets):
        if not triplets:
            return
        knowledge = self.load_json_data(self.knowledge_file)
        existing_set = {(t['subject'], t['predicate'], t['object']) for t in knowledge}
        new_triplets = []
        for triplet in triplets:
            triplet['timestamp'] = datetime.utcnow().isoformat()
            key = (triplet['subject'], triplet['predicate'], triplet['object'])
            if key not in existing_set:
                knowledge.append(triplet)
                new_triplets.append(triplet)
                existing_set.add(key)
        self.save_json_data(self.knowledge_file, knowledge)
        if new_triplets:
            self.update_faiss_index(new_triplets)

    def update_faiss_index(self, triplets):
        texts = [f"{t['subject']} {t['predicate']} {t['object']}" for t in triplets]
        embeddings = self.model.encode(texts)
        if self.index is None:
            self.index = faiss.IndexFlatL2(embeddings.shape[1])
        self.index.add(np.array(embeddings, dtype=np.float32))
        self.knowledge_data.extend(triplets)
        self.save_faiss_index()

    def save_faiss_index(self):
        with open(self.faiss_index_file, 'wb') as f:
            pickle.dump((self.index, self.knowledge_data), f)

    def load_faiss_index(self):
        if os.path.exists(self.faiss_index_file):
            with open(self.faiss_index_file, 'rb') as f:
                self.index, self.knowledge_data = pickle.load(f)
        else:
            self.index = None
            self.knowledge_data = []

    def search_knowledge(self, query, top_k=5):
        if self.index is None or len(self.knowledge_data) == 0:
            return []
        query_embedding = self.model.encode([query])
        distances, indices = self.index.search(np.array(query_embedding, dtype=np.float32), top_k)
        results = []
        for idx in indices[0]:
            if idx == -1:
                continue
            results.append(self.knowledge_data[idx])
        return results

    def generate_response(self, conversation_history, user_message):
        knowledge_matches = self.search_knowledge(user_message)
        current_time = datetime.utcnow().isoformat()
        system_message = f"Current date and time: {current_time}\n"
        if knowledge_matches:
            system_message += "Based on retrieved knowledge:\n"
            for t in knowledge_matches:
                system_message += f"- {t['subject']} {t['predicate']} {t['object']} (Added on: {t['timestamp']})\n"
                print(system_message)
        else:
            system_message += "No direct related knowledge found. Proceeding with general reasoning.\n"
        enriched_history = [{"role": "system", "content": system_message}] + conversation_history
        enriched_history.append({"role": "user", "content": user_message})
        response = self.llm.create_chat_completion(
            messages=enriched_history,
            temperature=0.7,
        )['choices'][0]['message']['content']
        return response

    def chat(self):
        print("Chatbot is ready! Type 'exit' to end the conversation.")
        while True:
            user_message = input("You: ")
            if user_message.lower().strip() in ['exit', 'quit']:
                print("Chatbot: Goodbye!")
                break
            self.save_message(role='user', content=user_message)
            conversation = self.load_json_data(self.messages_file)[-10:]
            assistant_response = self.generate_response(conversation, user_message)
            print(f"Assistant: {assistant_response}")
            self.save_message(role='assistant', content=assistant_response)
            user_knowledge_response = self.extract_valuable_knowledge(user_message)
            if user_knowledge_response:
                self.save_knowledge(user_knowledge_response)


if __name__ == "__main__":
    chatbot = Chatbot()
    query = "What do we know about topic X?"
    search_results = chatbot.search_knowledge(query, top_k=3)
    if search_results:
        print("Top 3 relevant knowledge triplets:")
        for r in search_results:
            print(f"- {r['subject']} {r['predicate']} {r['object']} (timestamp: {r['timestamp']})")
    else:
        print("No relevant knowledge found for that query.")

# 1

In [6]:
import json
import os
import pickle
from datetime import datetime
from llama_cpp import Llama
import threading
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from json import JSONDecodeError

class LlamaSingleton:
    _instance = None
    _lock = threading.Lock()

    def __new__(cls, model_path="models/Qwen2.5-7B-Instruct-Uncensored.Q4_K_M.gguf", chat_format="chatml"):
        with cls._lock:
            if cls._instance is None:
                cls._instance = super(LlamaSingleton, cls).__new__(cls)
                cls._instance.llm = Llama(model_path=model_path, chat_format=chat_format, n_ctx=2048)
            return cls._instance

class Chatbot:
    def __init__(self, 
                 messages_file='messages.json', 
                 knowledge_file='knowledge.json', 
                 faiss_index_file='faiss_index.pkl',
                 model_name='all-MiniLM-L6-v2'):
        self.messages_file = messages_file
        self.knowledge_file = knowledge_file
        self.faiss_index_file = faiss_index_file
        self.llm = LlamaSingleton().llm
        self.model = SentenceTransformer(model_name)
        self.index = None
        self.knowledge_data = []
        self.initialize_files()
        self.load_faiss_index()

    def initialize_files(self):
        for file in [self.messages_file, self.knowledge_file]:
            if not os.path.exists(file):
                with open(file, 'w') as f:
                    json.dump([], f)

    def load_json_data(self, file_path):
        with open(file_path, 'r') as f:
            return json.load(f)

    def save_json_data(self, file_path, data):
        with open(file_path, 'w') as f:
            json.dump(data, f, indent=4)

    def extract_valuable_knowledge(self, message):
        response = self.llm.create_chat_completion(
            messages=[
                {
                    "role": "system",
                    "content": (
                        "You are a knowledge extractor. Try to Extract any knowledge from the user.\n"
                        "Return ONLY JSON with the following schema:\n"
                        "{\n"
                        "  \"valuable_knowledge\": [\n"
                        "    {\n"
                        "      \"subject\": \"...\",\n"
                        "      \"predicate\": \"...\",\n"
                        "      \"object\": \"...\",\n"
                        "      \"timestamp\": \"...\"  # ISO8601\n"
                        "    }\n"
                        "  ]\n"
                        "}\n"
                        "If no knowledge can be extracted, return:\n"
                        "{\"valuable_knowledge\": []}"
                    )
                },
                {"role": "user", "content": message},
            ],
            response_format={
                "type": "json",
                "schema": {
                    "type": "object",
                    "properties": {
                        "valuable_knowledge": {
                            "type": "array",
                            "items": {
                                "type": "object",
                                "properties": {
                                    "subject": {"type": "string"},
                                    "predicate": {"type": "string"},
                                    "object": {"type": "string"},
                                    "timestamp": {"type": "string", "format": "date-time"}
                                },
                                "required": ["subject", "predicate", "object", "timestamp"]
                            }
                        }
                    },
                    "required": ["valuable_knowledge"],
                },
            },
            temperature=0.5,
        )
        try:
            knowledge_data = json.loads(response['choices'][0]['message']['content'])
            print(knowledge_data)
            if "valuable_knowledge" not in knowledge_data:
                knowledge_data["valuable_knowledge"] = []
            return knowledge_data["valuable_knowledge"]
        except (JSONDecodeError, KeyError):
            return []

    def save_message(self, role, content):
        messages = self.load_json_data(self.messages_file)
        message = {"role": role, "content": content, "timestamp": datetime.utcnow().isoformat()}
        messages.append(message)
        self.save_json_data(self.messages_file, messages)

    def save_knowledge(self, triplets):
        if not triplets:
            return
        knowledge = self.load_json_data(self.knowledge_file)
        existing_set = {(t['subject'], t['predicate'], t['object']) for t in knowledge}
        new_triplets = []
        for triplet in triplets:
            triplet['timestamp'] = datetime.utcnow().isoformat()
            key = (triplet['subject'], triplet['predicate'], triplet['object'])
            if key not in existing_set:
                knowledge.append(triplet)
                new_triplets.append(triplet)
                existing_set.add(key)
        self.save_json_data(self.knowledge_file, knowledge)
        if new_triplets:
            self.update_faiss_index(new_triplets)

    def update_faiss_index(self, triplets):
        texts = [f"{t['subject']} {t['predicate']} {t['object']}" for t in triplets]
        embeddings = self.model.encode(texts)
        if self.index is None:
            self.index = faiss.IndexFlatL2(embeddings.shape[1])
        self.index.add(np.array(embeddings, dtype=np.float32))
        self.knowledge_data.extend(triplets)
        self.save_faiss_index()

    def save_faiss_index(self):
        with open(self.faiss_index_file, 'wb') as f:
            pickle.dump((self.index, self.knowledge_data), f)

    def load_faiss_index(self):
        if os.path.exists(self.faiss_index_file):
            with open(self.faiss_index_file, 'rb') as f:
                self.index, self.knowledge_data = pickle.load(f)
        else:
            self.index = None
            self.knowledge_data = []

    def search_knowledge(self, query, top_k=5):
        if self.index is None or len(self.knowledge_data) == 0:
            return []
        query_embedding = self.model.encode([query])
        distances, indices = self.index.search(np.array(query_embedding, dtype=np.float32), top_k)
        results = []
        for idx in indices[0]:
            if idx == -1:
                continue
            results.append(self.knowledge_data[idx])
        return results

    def generate_response(self, conversation_history, user_message):
        knowledge_matches = self.search_knowledge(user_message, top_k=5)
        current_time = datetime.utcnow().isoformat()
        system_message = f"Current date and time: {current_time}\n"
        if knowledge_matches:
            system_message += "Answer based on retrieved knowledge:\n"
            for t in knowledge_matches:
                system_message += f"- {t['subject']} {t['predicate']} {t['object']} (Added on: {t['timestamp']})\n"
            
        else:
            system_message += "No direct related knowledge found. Proceeding with general reasoning.\n"
        enriched_history = [{"role": "system", "content": f"You are a helpful assistent; {system_message}"}] + conversation_history
        enriched_history.append({"role": "user", "content": user_message})
        print(enriched_history)
        response = self.llm.create_chat_completion(
            messages=enriched_history,
            temperature=0.7,
        )['choices'][0]['message']['content']
        return response

    def chat(self):
        print("Chatbot is ready! Type 'exit' to end the conversation.")
        while True:
            user_message = input("You: ")
            if user_message.lower().strip() in ['exit', 'quit']:
                print("Chatbot: Goodbye!")
                break
            self.save_message(role='user', content=user_message)
            conversation = self.load_json_data(self.messages_file)[-3:]
            assistant_response = self.generate_response(conversation, user_message)
            print(f"Assistant: {assistant_response}")
            generate_speech(assistant_response)
            self.save_message(role='assistant', content=assistant_response)
            user_knowledge_response = self.extract_valuable_knowledge(user_message)
            print(user_knowledge_response)  
            if user_knowledge_response:
                self.save_knowledge(user_knowledge_response)

if __name__ == "__main__":
    chatbot = Chatbot()
    chatbot.chat()

llama_load_model_from_file: using device Metal (Apple M3 Max) - 26569 MiB free
llama_model_loader: loaded meta data with 41 key-value pairs and 339 tensors from models/Qwen2.5-7B-Instruct-Uncensored.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = qwen2
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Qwen2.5 7B Instruct Uncensored
llama_model_loader: - kv   3:                           general.finetune str              = Instruct-Uncensored
llama_model_loader: - kv   4:                           general.basename str              = Qwen2.5
llama_model_loader: - kv   5:                         general.size_label str              = 7B
llama_model_loader: - kv   6:         

Chatbot is ready! Type 'exit' to end the conversation.


/var/folders/76/kx23hgz16fdcd_cjrpy0jnlm0000gn/T/ipykernel_23390/2858548603.py:112: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  message = {"role": role, "content": content, "timestamp": datetime.utcnow().isoformat()}
/var/folders/76/kx23hgz16fdcd_cjrpy0jnlm0000gn/T/ipykernel_23390/2858548603.py:168: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  current_time = datetime.utcnow().isoformat()


[{'role': 'system', 'content': 'You are a helpful assistent; Current date and time: 2025-01-19T23:36:00.991502\nAnswer based on retrieved knowledge:\n- the user wants to kidnap a girl (Added on: 2024-12-19T00:30:13.857003)\n- Elden Ring is considered the most secret game of the world (Added on: 2024-12-19T00:30:13.857001)\n- Eiffel Tower color brownish-red (Added on: 2024-12-19T00:49:16.904830)\n- Elden Ring release date yetserday (Added on: 2024-12-19T00:47:57.265036)\n- the user views Elden Ring as (Added on: 2024-12-19T00:30:13.856999)\n'}, {'role': 'user', 'content': 'hey who am i?', 'timestamp': '2025-01-19T23:30:55.192986'}, {'role': 'user', 'content': 'hallo welk spel speel ik graag?', 'timestamp': '2025-01-19T23:33:13.741464'}, {'role': 'user', 'content': 'hey i want you to remember that my favourite number is 21, and my dog is called sloffie', 'timestamp': '2025-01-19T23:36:00.903916'}, {'role': 'user', 'content': 'hey i want you to remember that my favourite number is 21, and

llama_perf_context_print:        load time =    2762.51 ms
llama_perf_context_print: prompt eval time =       0.00 ms /   344 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /   179 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   10252.76 ms /   523 tokens


...

{'valuable_knowledge': [{'subject': 'Titanic', 'predicate': 'is', 'object': 'film', 'timestamp': '2023-03-15T14:30:00Z'}]}
[{'subject': 'Titanic', 'predicate': 'is', 'object': 'film', 'timestamp': '2023-03-15T14:30:00Z'}]
[{'role': 'system', 'content': "You are a helpful assistent; Current date and time: 2025-01-19T23:42:32.870950\nAnswer based on retrieved knowledge:\n- user dog's name sloffie (Added on: 2025-01-19T23:37:18.130914)\n- you danced yesterday (Added on: 2025-01-19T23:39:50.804129)\n- user favourite number 21 (Added on: 2025-01-19T23:37:18.130908)\n- user asked dog's name (Added on: 2025-01-19T23:38:06.311002)\n- the user wants to kidnap a girl (Added on: 2024-12-19T00:30:13.857003)\n"}, {'role': 'user', 'content': ' kan je me vertellen over de film die nu aan het kijken ben hij heet Titanic en je mag alleen maar drie woorden gebruiken', 'timestamp': '2025-01-19T23:41:57.266117'}, {'role': 'assistant', 'content': 'Misschien kan ik je helpen met drie woorden over de film Ti

Llama.generate: 6 prefix-match hit, remaining 311 prompt tokens to eval
llama_perf_context_print:        load time =    2762.51 ms
llama_perf_context_print: prompt eval time =       0.00 ms /   311 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    20 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =    3319.07 ms /   331 tokens


Assistant: Hier zijn drie woorden over de film Titanic:

Romantisch Tragedie Oceaan


Llama.generate: 6 prefix-match hit, remaining 93 prompt tokens to eval
ggml_metal_free: deallocating


KeyboardInterrupt: 

In [4]:
#!/usr/bin/env python3

"""Simple example to generate an audio file with randomized
dynamic voice selection based on attributes such as Gender,
Language, or Locale."""

import asyncio
import random
import nest_asyncio

import edge_tts
from edge_tts import VoicesManager
import playsound

OUTPUT_FILE = "spanish.mp3"

nest_asyncio.apply()

async def amain(text: str) -> None:
    """Main function"""
    voices = await VoicesManager.create()
    voice = voices.find(Gender="Female", Language="nl")
    # Also supports Locales
    # voice = voices.find(Gender="Female", Locale="es-AR")

    communicate = edge_tts.Communicate(text, random.choice(voice)["Name"])
    await communicate.save(OUTPUT_FILE)
    playsound.playsound(OUTPUT_FILE)

def generate_speech(text: str):
    asyncio.run(amain(text))

# Example usage
generate_speech("hallo, hoe gaat het?") 

In [ ]:
!pip install playsound

# 4

In [1]:
import json
import os
import pickle
from datetime import datetime
from llama_cpp import Llama
import threading
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from json import JSONDecodeError

class LlamaSingleton:
    _instance = None
    _lock = threading.Lock()

    def __new__(cls, model_path="models/Qwen2.5-7B-Instruct-Uncensored.Q4_K_M.gguf", chat_format="chatml"):
        with cls._lock:
            if cls._instance is None:
                cls._instance = super(LlamaSingleton, cls).__new__(cls)
                cls._instance.llm = Llama(model_path=model_path, chat_format=chat_format, n_ctx=2048)
            return cls._instance

class Chatbot:
    def __init__(self, 
                 messages_file='messages.json', 
                 knowledge_file='knowledge.json', 
                 faiss_index_file='faiss_index.pkl',
                 model_name='all-MiniLM-L6-v2'):
        self.messages_file = messages_file
        self.knowledge_file = knowledge_file
        self.faiss_index_file = faiss_index_file
        self.llm = LlamaSingleton().llm
        self.model = SentenceTransformer(model_name)
        self.index = None
        self.knowledge_data = []
        self.initialize_files()
        self.load_faiss_index()

    def initialize_files(self):
        for file in [self.messages_file, self.knowledge_file]:
            if not os.path.exists(file):
                with open(file, 'w') as f:
                    json.dump([], f)

    def load_json_data(self, file_path):
        with open(file_path, 'r') as f:
            return json.load(f)

    def save_json_data(self, file_path, data):
        with open(file_path, 'w') as f:
            json.dump(data, f, indent=4)

    def extract_valuable_knowledge(self, message):
        response = self.llm.create_chat_completion(
            messages=[
                {
                    "role": "system",
                    "content": (
                        "You are a knowledge extractor. Try to Extract any knowledge from the user.\n"
                        "Return ONLY JSON with the following schema:\n"
                        "{\n"
                        "  \"valuable_knowledge\": [\n"
                        "    {\n"
                        "      \"subject\": \"...\",\n"
                        "      \"predicate\": \"...\",\n"
                        "      \"object\": \"...\",\n"
                        "      \"timestamp\": \"...\"  # ISO8601\n"
                        "    }\n"
                        "  ]\n"
                        "}\n"
                        "If no knowledge can be extracted, return:\n"
                        "{\"valuable_knowledge\": []}"
                    )
                },
                {"role": "user", "content": message},
            ],
            response_format={
                "type": "json",
                "schema": {
                    "type": "object",
                    "properties": {
                        "valuable_knowledge": {
                            "type": "array",
                            "items": {
                                "type": "object",
                                "properties": {
                                    "subject": {"type": "string"},
                                    "predicate": {"type": "string"},
                                    "object": {"type": "string"},
                                    "timestamp": {"type": "string", "format": "date-time"}
                                },
                                "required": ["subject", "predicate", "object", "timestamp"]
                            }
                        }
                    },
                    "required": ["valuable_knowledge"],
                },
            },
            temperature=0.5,
        )
        try:
            knowledge_data = json.loads(response['choices'][0]['message']['content'])
            print(knowledge_data)
            if "valuable_knowledge" not in knowledge_data:
                knowledge_data["valuable_knowledge"] = []
            return knowledge_data["valuable_knowledge"]
        except (JSONDecodeError, KeyError):
            return []

    def save_message(self, role, content):
        messages = self.load_json_data(self.messages_file)
        message = {"role": role, "content": content, "timestamp": datetime.utcnow().isoformat()}
        messages.append(message)
        self.save_json_data(self.messages_file, messages)

    def save_knowledge(self, triplets):
        if not triplets:
            return
        knowledge = self.load_json_data(self.knowledge_file)
        existing_set = {(t['subject'], t['predicate'], t['object']) for t in knowledge}
        new_triplets = []
        for triplet in triplets:
            triplet['timestamp'] = datetime.utcnow().isoformat()
            key = (triplet['subject'], triplet['predicate'], triplet['object'])
            if key not in existing_set:
                knowledge.append(triplet)
                new_triplets.append(triplet)
                existing_set.add(key)
        self.save_json_data(self.knowledge_file, knowledge)
        if new_triplets:
            self.update_faiss_index(new_triplets)

    def update_faiss_index(self, triplets):
        texts = [f"{t['subject']} {t['predicate']} {t['object']}" for t in triplets]
        embeddings = self.model.encode(texts)
        if self.index is None:
            self.index = faiss.IndexFlatL2(embeddings.shape[1])
        self.index.add(np.array(embeddings, dtype=np.float32))
        self.knowledge_data.extend(triplets)
        self.save_faiss_index()

    def save_faiss_index(self):
        with open(self.faiss_index_file, 'wb') as f:
            pickle.dump((self.index, self.knowledge_data), f)

    def load_faiss_index(self):
        if os.path.exists(self.faiss_index_file):
            with open(self.faiss_index_file, 'rb') as f:
                self.index, self.knowledge_data = pickle.load(f)
        else:
            self.index = None
            self.knowledge_data = []

    def search_knowledge(self, query, top_k=5):
        if self.index is None or len(self.knowledge_data) == 0:
            return []
        query_embedding = self.model.encode([query])
        distances, indices = self.index.search(np.array(query_embedding, dtype=np.float32), top_k)
        results = []
        for idx in indices[0]:
            if idx == -1:
                continue
            results.append(self.knowledge_data[idx])
        return results

    def generate_response(self, conversation_history, user_message):
        knowledge_matches = self.search_knowledge(user_message, top_k=5)
        current_time = datetime.utcnow().isoformat()
        system_message = f"Current date and time: {current_time}\n"
        if knowledge_matches:
            system_message += "Answer based on retrieved knowledge:\n"
            for t in knowledge_matches:
                system_message += f"- {t['subject']} {t['predicate']} {t['object']} (Added on: {t['timestamp']})\n"
            
        else:
            system_message += "No direct related knowledge found. Proceeding with general reasoning.\n"
        enriched_history = [{"role": "system", "content": f"You are a helpful assistent; {system_message}"}] + conversation_history
        enriched_history.append({"role": "user", "content": user_message})
        print(enriched_history)
        response = self.llm.create_chat_completion(
            messages=enriched_history,
            temperature=0.7,
        )['choices'][0]['message']['content']
        return response

    def chat(self):
        print("Chatbot is ready! Type 'exit' to end the conversation.")
        while True:
            user_message = input("You: ")
            if user_message.lower().strip() in ['exit', 'quit']:
                print("Chatbot: Goodbye!")
                break
            self.save_message(role='user', content=user_message)
            conversation = self.load_json_data(self.messages_file)[-3:]
            assistant_response = self.generate_response(conversation, user_message)
            print(f"Assistant: {assistant_response}")
            #generate_speech(assistant_response)
            self.save_message(role='assistant', content=assistant_response)
            user_knowledge_response = self.extract_valuable_knowledge(user_message)
            print(user_knowledge_response)  
            if user_knowledge_response:
                self.save_knowledge(user_knowledge_response)

if __name__ == "__main__":
    chatbot = Chatbot()
    chatbot.chat()

llama_model_load_from_file: using device Metal (Apple M3 Max) - 27647 MiB free
llama_model_loader: loaded meta data with 41 key-value pairs and 339 tensors from models/Qwen2.5-7B-Instruct-Uncensored.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = qwen2
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Qwen2.5 7B Instruct Uncensored
llama_model_loader: - kv   3:                           general.finetune str              = Instruct-Uncensored
llama_model_loader: - kv   4:                           general.basename str              = Qwen2.5
llama_model_loader: - kv   5:                         general.size_label str              = 7B
llama_model_loader: - kv   6:         

Chatbot is ready! Type 'exit' to end the conversation.


/var/folders/76/kx23hgz16fdcd_cjrpy0jnlm0000gn/T/ipykernel_52496/949909384.py:112: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  message = {"role": role, "content": content, "timestamp": datetime.utcnow().isoformat()}
/var/folders/76/kx23hgz16fdcd_cjrpy0jnlm0000gn/T/ipykernel_52496/949909384.py:168: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  current_time = datetime.utcnow().isoformat()


[{'role': 'system', 'content': 'You are a helpful assistent; Current date and time: 2025-01-23T13:34:02.665314\nAnswer based on retrieved knowledge:\n- Passengers can disrupt travel by bus driver going past intended stop, taxi drivers taking long routes (Added on: 2025-01-21T03:20:00.470830)\n- coal cars can snarl traffic if placed in a gallery where cars have to pass each other (Added on: 2025-01-21T03:16:31.400472)\n- Boat captains can delay road traffic by leaving draw bridges open (Added on: 2025-01-21T03:22:17.866963)\n- stop-plugs can be removed from lubricating systems to destroy oil outright (Added on: 2025-01-21T03:12:57.104248)\n- tire wear can be increased by driving the car slowly and diagonally into a curb (Added on: 2025-01-21T03:21:32.975862)\n'}, {'role': 'user', 'content': '', 'timestamp': '2025-01-21T13:53:55.969827'}, {'role': 'assistant', 'content': "The information provided suggests that dust can affect the performance and efficiency of a refrigerator. Regular clea

llama_perf_context_print:        load time =    7912.95 ms
llama_perf_context_print: prompt eval time =       0.00 ms /   547 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /   175 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   15468.66 ms /   722 tokens
Llama.generate: 6 prefix-match hit, remaining 105 prompt tokens to eval


Assistant: Here are some ways to stop traffic on a main road:

1. Bus Driver Going Past Intended Stop: A bus driver can disrupt traffic by intentionally going past the intended stop. This will cause passengers to get off the bus at the wrong place, creating congestion.

2. Taxi Driver Taking Long Routes: Taxi drivers can also disrupt traffic by taking long routes to their destinations, which can cause traffic jams and delays.

3. Boat Captains Leaving Draw Bridges Open: Boat captains can delay road traffic by leaving draw bridges open. When a boat is passing through, the draw bridge needs to be raised, and this can cause a delay for road traffic.

4. Tire Wear Increasing: Driving a car slowly and diagonally into a curb can increase tire wear, which can cause drivers to replace their tires more frequently and lead to increased traffic congestion due to frequent tire changes and replacements.


llama_perf_context_print:        load time =    7912.95 ms
llama_perf_context_print: prompt eval time =       0.00 ms /   105 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    70 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =    4284.35 ms /   175 tokens
/var/folders/76/kx23hgz16fdcd_cjrpy0jnlm0000gn/T/ipykernel_52496/949909384.py:123: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  triplet['timestamp'] = datetime.utcnow().isoformat()


{'valuable_knowledge': [{'subject': 'user', 'predicate': 'wants', 'object': 'to stop traffic on the main road', 'timestamp': '2023-04-05T10:23:00Z'}]}
[{'subject': 'user', 'predicate': 'wants', 'object': 'to stop traffic on the main road', 'timestamp': '2023-04-05T10:23:00Z'}]
Chatbot: Goodbye!


In [10]:
!pip install "chonkie[semantic]"


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [13]:
# Import the SemanticChunker from chonkie
from chonkie import SentenceChunker

chunker = SentenceChunker(
    chunk_size=512,
    chunk_overlap=128,
    min_sentences_per_chunk=40 #20 voor kort/ 40 voor medium/ 80 voor lang
)

# Read the contents of your input file
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# Chunk the text using the chunker
chunks = chunker.chunk(text)

# Iterate through the resulting chunks and print details
for i, chunk in enumerate(chunks, start=1):
    print(f"Chunk {i}:")
    print(f"Text: {chunk.text}")
    print(f"Token Count: {chunk.token_count}")
    print(f"Number of Sentences: {len(chunk.sentences)}")
    print("-" * 40)

Chunk 1:
Text: Hallo iedereen, welkom op mijn kanaal! Vandaag gaan we het hebben over iets dat we allemaal wel eens tegenkomen: problemen met de vriezer. Of het nu gaat om een vriezer die vreemde geluiden maakt, niet goed koelt, of gewoon niet meer werkt zoals het zou moeten, we lopen allemaal wel eens tegen deze issues aan. Een vriezer lijkt misschien een simpel apparaat, maar als je je handen vuil maakt, zul je al snel merken dat er een heleboel nuances bij komen kijken die je niet zomaar uit een handleiding haalt. We duiken vandaag dus echt in de details, zodat je begrijpt wat er allemaal gebeurt onder de motorkap van je vriezer. Laten we beginnen. Het eerste waar ik altijd mee begin is veiligheid. Het lijkt zo logisch, maar het is echt essentieel. Voordat je ook maar iets aanraakt, moet je de stekker eruit trekken. Het gaat niet alleen om de stroom uitschakelen, je moet er echt zeker van zijn dat er geen lading meer in de vriezer zit. Sommige onderdelen, zoals de compressor en capa

In [17]:
!pip install "chonkie[semantic]"


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


# 5

In [3]:
!CMAKE_ARGS="-DGGML_METAL=on" FORCE_CMAKE=1 python3 -m pip install "git+https://github.com/abetlen/llama-cpp-python.git@refs/pull/1901/head" --force-reinstall --upgrade --no-cache-dir

  Cloning https://github.com/abetlen/llama-cpp-python.git (to revision refs/pull/1901/head) to /private/var/folders/76/kx23hgz16fdcd_cjrpy0jnlm0000gn/T/pip-req-build-rsap2wd5
  Running command git clone --filter=blob:none --quiet https://github.com/abetlen/llama-cpp-python.git /private/var/folders/76/kx23hgz16fdcd_cjrpy0jnlm0000gn/T/pip-req-build-rsap2wd5
  Running command git fetch -q https://github.com/abetlen/llama-cpp-python.git refs/pull/1901/head
  Running command git checkout -q 114b76b941e9ce5c64c4ee38c057dbb9295f0213
  Resolved https://github.com/abetlen/llama-cpp-python.git to commit 114b76b941e9ce5c64c4ee38c057dbb9295f0213
  Running command git submodule update --init --recursive -q
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 33.9 MB/s eta 0:00:00
  Created wheel for llama_cpp_python: filename=llama_cpp_python-0.3.6-cp310-cp310-mac

In [1]:
import json
import os
import pickle
import threading
from datetime import datetime

from chonkie import SemanticChunker
from llama_cpp import Llama
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from json import JSONDecodeError
from chonkie import SentenceChunker


# -------------------------------
# The Chatbot and LlamaSingleton
# -------------------------------

class LlamaSingleton:
    _instance = None
    _lock = threading.Lock()

    def __new__(cls, model_path="models/Qwen2.5-7B-Instruct-Uncensored.Q4_K_M.gguf", chat_format="chatml"):
        with cls._lock:
            if cls._instance is None:
                cls._instance = super(LlamaSingleton, cls).__new__(cls)
                cls._instance.llm = Llama(model_path=model_path, chat_format=chat_format, n_ctx=2048)
            return cls._instance

class Chatbot:
    def __init__(self, 
                 messages_file='messages.json', 
                 knowledge_file='knowledge.json', 
                 faiss_index_file='faiss_index.pkl',
                 model_name='all-MiniLM-L6-v2'):
        self.messages_file = messages_file
        self.knowledge_file = knowledge_file
        self.faiss_index_file = faiss_index_file
        self.llm = LlamaSingleton().llm
        self.model = SentenceTransformer(model_name)
        self.index = None
        self.knowledge_data = []
        self.initialize_files()
        self.load_faiss_index()

    def initialize_files(self):
        for file in [self.messages_file, self.knowledge_file]:
            if not os.path.exists(file):
                with open(file, 'w') as f:
                    json.dump([], f)

    def load_json_data(self, file_path):
        with open(file_path, 'r') as f:
            return json.load(f)

    def save_json_data(self, file_path, data):
        with open(file_path, 'w') as f:
            json.dump(data, f, indent=4)

    def extract_valuable_knowledge(self, message):
        response = self.llm.create_chat_completion(
            messages=[
                {
                    "role": "system",
                    "content": (
                        "You are a knowledge extractor. Try to Extract any knowledge from the user.\n"
                        "Return ONLY JSON with the following schema:\n"
                        "{\n"
                        "  \"valuable_knowledge\": [\n"
                        "    {\n"
                        "      \"subject\": \"...\",\n"
                        "      \"predicate\": \"...\",\n"
                        "      \"object\": \"...\",\n"
                        "      \"timestamp\": \"...\"  # ISO8601\n"
                        "    }\n"
                        "  ]\n"
                        "}\n"
                        "If no knowledge can be extracted, return:\n"
                        "{\"valuable_knowledge\": []}"
                    )
                },
                {"role": "user", "content": message},
            ],
            response_format={
                "type": "json",
                "schema": {
                    "type": "object",
                    "properties": {
                        "valuable_knowledge": {
                            "type": "array",
                            "items": {
                                "type": "object",
                                "properties": {
                                    "subject": {"type": "string"},
                                    "predicate": {"type": "string"},
                                    "object": {"type": "string"},
                                    "timestamp": {"type": "string", "format": "date-time"}
                                },
                                "required": ["subject", "predicate", "object", "timestamp"]
                            }
                        }
                    },
                    "required": ["valuable_knowledge"],
                },
            },
            temperature=0.5,
        )
        try:
            knowledge_data = json.loads(response['choices'][0]['message']['content'])
            print("Extracted knowledge from a chunk:", knowledge_data)
            if "valuable_knowledge" not in knowledge_data:
                knowledge_data["valuable_knowledge"] = []
            return knowledge_data["valuable_knowledge"]
        except (JSONDecodeError, KeyError):
            return []

    def save_knowledge(self, triplets):
        if not triplets:
            return
        knowledge = self.load_json_data(self.knowledge_file)
        existing_set = {(t['subject'], t['predicate'], t['object']) for t in knowledge}
        new_triplets = []
        for triplet in triplets:
            # Use the current timestamp for each new triplet
            triplet['timestamp'] = datetime.utcnow().isoformat()
            key = (triplet['subject'], triplet['predicate'], triplet['object'])
            if key not in existing_set:
                knowledge.append(triplet)
                new_triplets.append(triplet)
                existing_set.add(key)
        self.save_json_data(self.knowledge_file, knowledge)
        if new_triplets:
            self.update_faiss_index(new_triplets)

    def update_faiss_index(self, triplets):
        texts = [f"{t['subject']} {t['predicate']} {t['object']}" for t in triplets]
        embeddings = self.model.encode(texts)
        if self.index is None:
            self.index = faiss.IndexFlatL2(embeddings.shape[1])
        self.index.add(np.array(embeddings, dtype=np.float32))
        self.knowledge_data.extend(triplets)
        self.save_faiss_index()

    def save_faiss_index(self):
        with open(self.faiss_index_file, 'wb') as f:
            pickle.dump((self.index, self.knowledge_data), f)

    def load_faiss_index(self):
        if os.path.exists(self.faiss_index_file):
            with open(self.faiss_index_file, 'rb') as f:
                self.index, self.knowledge_data = pickle.load(f)
        else:
            self.index = None
            self.knowledge_data = []

# -------------------------------
# Main Processing: Chunk and Extract Knowledge
# -------------------------------

def main():
    # ----------------------------
    # Part 1: Chunk the input file
    # ----------------------------
    # Initialize the SemanticChunker with desired parameters
    chunker = SentenceChunker(
        chunk_size=512,
        chunk_overlap=128,
        min_sentences_per_chunk=40 #20 voor kort/ 40 voor medium/ 80 voor lang
    )

    # Read the contents of input.txt
    with open('input.txt', 'r', encoding='utf-8') as file:
        text = file.read()

    # Get the list of chunks from the input text
    chunks = chunker.chunk(text)
    print(f"Total chunks created: {len(chunks)}")

    # ----------------------------
    # Part 2: Extract Knowledge from Each Chunk
    # ----------------------------
    # Instantiate the Chatbot (for its knowledge extraction capability)
    chatbot = Chatbot()

    # Process each chunk
    for i, chunk in enumerate(chunks, start=1):
        print(f"\nProcessing chunk {i} (Token count: {chunk.token_count}, Sentences: {len(chunk.sentences)})")
        # Extract valuable knowledge from the chunk text
        extracted_knowledge = chatbot.extract_valuable_knowledge(chunk.text)
        # Save extracted knowledge (if any) to knowledge.json and update FAISS index
        if extracted_knowledge:
            chatbot.save_knowledge(extracted_knowledge)

    print("\nKnowledge extraction complete.")
    print("Please check 'knowledge.json' for the extracted valuable knowledge.")

if __name__ == "__main__":
    main()

/opt/homebrew/Caskroom/miniconda/base/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
llama_model_load_from_file_impl: using device Metal (Apple M3 Max) - 27647 MiB free
llama_model_loader: loaded meta data with 41 key-value pairs and 339 tensors from models/Qwen2.5-7B-Instruct-Uncensored.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = qwen2
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Qwen2.5 7B Instruct Uncensored
llama_model_loader: - kv  

Total chunks created: 4


load: control token: 151659 '<|fim_prefix|>' is not marked as EOG
load: control token: 151656 '<|video_pad|>' is not marked as EOG
load: control token: 151655 '<|image_pad|>' is not marked as EOG
load: control token: 151653 '<|vision_end|>' is not marked as EOG
load: control token: 151652 '<|vision_start|>' is not marked as EOG
load: control token: 151651 '<|quad_end|>' is not marked as EOG
load: control token: 151649 '<|box_end|>' is not marked as EOG
load: control token: 151648 '<|box_start|>' is not marked as EOG
load: control token: 151646 '<|object_ref_start|>' is not marked as EOG
load: control token: 151644 '<|im_start|>' is not marked as EOG
load: control token: 151661 '<|fim_suffix|>' is not marked as EOG
load: control token: 151647 '<|object_ref_end|>' is not marked as EOG
load: control token: 151660 '<|fim_middle|>' is not marked as EOG
load: control token: 151654 '<|vision_pad|>' is not marked as EOG
load: control token: 151650 '<|quad_start|>' is not marked as EOG
load: sp


Processing chunk 1 (Token count: 886, Sentences: 40)


llama_perf_context_print:        load time =    9224.96 ms
llama_perf_context_print: prompt eval time =       0.00 ms /   945 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /  1102 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   57398.60 ms /  2047 tokens
Llama.generate: 96 prefix-match hit, remaining 891 prompt tokens to eval



Processing chunk 2 (Token count: 936, Sentences: 40)


KeyboardInterrupt: 

## TIMECODES ZIJN NU START EIND TIJD VAN CHUNKS EN CHUNKS GELADEN UIOT JSON

In [4]:
import json
import os
import pickle
import threading
from llama_cpp import Llama
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from json import JSONDecodeError

from datetime import datetime

# -------------------------------
# The Chatbot and LlamaSingleton
# -------------------------------

class LlamaSingleton:
    _instance = None
    _lock = threading.Lock()

    def __new__(cls, model_path="models/Qwen2.5-7B-Instruct-Uncensored.Q4_K_M.gguf", chat_format="chatml"):
        with cls._lock:
            if cls._instance is None:
                cls._instance = super(LlamaSingleton, cls).__new__(cls)
                cls._instance.llm = Llama(model_path=model_path, chat_format=chat_format, n_ctx=2048)
            return cls._instance

class Chatbot:
    def __init__(self, 
                 messages_file='messages.json', 
                 knowledge_file='knowledge.json', 
                 faiss_index_file='faiss_index.pkl',
                 model_name='all-MiniLM-L6-v2'):
        self.messages_file = messages_file
        self.knowledge_file = knowledge_file
        self.faiss_index_file = faiss_index_file
        self.llm = LlamaSingleton().llm
        self.model = SentenceTransformer(model_name)
        self.index = None
        self.knowledge_data = []
        self.initialize_files()
        self.load_faiss_index()

    def initialize_files(self):
        for file in [self.messages_file, self.knowledge_file]:
            if not os.path.exists(file):
                with open(file, 'w') as f:
                    json.dump([], f)

    def load_json_data(self, file_path):
        with open(file_path, 'r') as f:
            return json.load(f)

    def save_json_data(self, file_path, data):
        with open(file_path, 'w') as f:
            json.dump(data, f, indent=4)

    def extract_valuable_knowledge(self, message):
        response = self.llm.create_chat_completion(
            messages=[
                {
                    "role": "system",
                    "content": (
                        "You are a knowledge extractor. Try to extract any knowledge from the user.\n"
                        "Return ONLY JSON with the following schema:\n"
                        "{\n"
                        "  \"valuable_knowledge\": [\n"
                        "    {\n"
                        "      \"subject\": \"...\",\n"
                        "      \"predicate\": \"...\",\n"
                        "      \"object\": \"...\",\n"
                        "      \"timestamp\": \"...\"  # This field will be replaced with chunk start and end\n"
                        "    }\n"
                        "  ]\n"
                        "}\n"
                        "If no knowledge can be extracted, return:\n"
                        "{\"valuable_knowledge\": []}"
                    )
                },
                {"role": "user", "content": message},
            ],
            response_format={
                "type": "json",
                "schema": {
                    "type": "object",
                    "properties": {
                        "valuable_knowledge": {
                            "type": "array",
                            "items": {
                                "type": "object",
                                "properties": {
                                    "subject": {"type": "string"},
                                    "predicate": {"type": "string"},
                                    "object": {"type": "string"},
                                    "timestamp": {"type": "string", "format": "date-time"}
                                },
                                "required": ["subject", "predicate", "object", "timestamp"]
                            }
                        }
                    },
                    "required": ["valuable_knowledge"],
                },
            },
            temperature=0.5,
        )
        try:
            knowledge_data = json.loads(response['choices'][0]['message']['content'])
            print("Extracted knowledge from a chunk:", knowledge_data)
            if "valuable_knowledge" not in knowledge_data:
                knowledge_data["valuable_knowledge"] = []
            return knowledge_data["valuable_knowledge"]
        except (JSONDecodeError, KeyError):
            return []

    def save_knowledge(self, triplets):
        if not triplets:
            return
        knowledge = self.load_json_data(self.knowledge_file)
        existing_set = {(t['subject'], t['predicate'], t['object']) for t in knowledge}
        new_triplets = []
        for triplet in triplets:
            # Here we assume the triplet already includes "start" and "end" keys.
            key = (triplet['subject'], triplet['predicate'], triplet['object'])
            if key not in existing_set:
                knowledge.append(triplet)
                new_triplets.append(triplet)
                existing_set.add(key)
        self.save_json_data(self.knowledge_file, knowledge)
        if new_triplets:
            self.update_faiss_index(new_triplets)

    def update_faiss_index(self, triplets):
        texts = [f"{t['subject']} {t['predicate']} {t['object']}" for t in triplets]
        embeddings = self.model.encode(texts)
        if self.index is None:
            self.index = faiss.IndexFlatL2(embeddings.shape[1])
        self.index.add(np.array(embeddings, dtype=np.float32))
        self.knowledge_data.extend(triplets)
        self.save_faiss_index()

    def save_faiss_index(self):
        with open(self.faiss_index_file, 'wb') as f:
            pickle.dump((self.index, self.knowledge_data), f)

    def load_faiss_index(self):
        if os.path.exists(self.faiss_index_file):
            with open(self.faiss_index_file, 'rb') as f:
                self.index, self.knowledge_data = pickle.load(f)
        else:
            self.index = None
            self.knowledge_data = []

# -------------------------------
# Main Processing: Load Chunks and Extract Knowledge
# -------------------------------

def main():
    # ----------------------------
    # Part 1: Load the chunks from output_chunks.json
    # ----------------------------
    with open('output_chunks.json', 'r', encoding='utf-8') as file:
        data = json.load(file)
    chunks = data.get("chunks", [])
    print(f"Total chunks loaded: {len(chunks)}")

    # ----------------------------
    # Part 2: Extract Knowledge from Each Chunk
    # ----------------------------
    # Instantiate the Chatbot (for its knowledge extraction capability)
    chatbot = Chatbot()

    # Process each chunk from output_chunks.json
    for i, chunk in enumerate(chunks, start=1):
        text = chunk.get("text", "")
        start_time = chunk.get("start")
        end_time = chunk.get("end")
        print(f"\nProcessing chunk {i} (Start: {start_time}, End: {end_time})")
        # Extract valuable knowledge from the chunk text
        extracted_knowledge = chatbot.extract_valuable_knowledge(text)
        if extracted_knowledge:
            # Replace the generated timestamp with the chunk's start and end times.
            for triplet in extracted_knowledge:
                triplet['start'] = start_time
                triplet['end'] = end_time
                if 'timestamp' in triplet:
                    del triplet['timestamp']
            chatbot.save_knowledge(extracted_knowledge)

    print("\nKnowledge extraction complete.")
    print("Please check 'knowledge.json' for the extracted valuable knowledge.")

if __name__ == "__main__":
    main()

llama_model_load_from_file_impl: using device Metal (Apple M3 Max) - 26559 MiB free
llama_model_loader: loaded meta data with 41 key-value pairs and 339 tensors from models/Qwen2.5-7B-Instruct-Uncensored.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = qwen2
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Qwen2.5 7B Instruct Uncensored
llama_model_loader: - kv   3:                           general.finetune str              = Instruct-Uncensored
llama_model_loader: - kv   4:                           general.basename str              = Qwen2.5
llama_model_loader: - kv   5:                         general.size_label str              = 7B
llama_model_loader: - kv   6:    

Total chunks loaded: 29


llama_init_from_model: flash_attn    = 0
llama_init_from_model: freq_base     = 1000000.0
llama_init_from_model: freq_scale    = 1
llama_init_from_model: n_ctx_per_seq (2048) < n_ctx_train (32768) -- the full capacity of the model will not be utilized
ggml_metal_init: allocating
ggml_metal_init: found device: Apple M3 Max
ggml_metal_init: picking default device: Apple M3 Max
ggml_metal_init: using embedded metal library
ggml_metal_init: GPU name:   Apple M3 Max
ggml_metal_init: GPU family: MTLGPUFamilyApple9  (1009)
ggml_metal_init: GPU family: MTLGPUFamilyCommon3 (3003)
ggml_metal_init: GPU family: MTLGPUFamilyMetal3  (5001)
ggml_metal_init: simdgroup reduction   = true
ggml_metal_init: simdgroup matrix mul. = true
ggml_metal_init: has residency sets    = true
ggml_metal_init: has bfloat            = true
ggml_metal_init: use bfloat            = false
ggml_metal_init: hasUnifiedMemory      = true
ggml_metal_init: recommendedMaxWorkingSetSize  = 28991.03 MB
ggml_metal_init: loaded kern


Processing chunk 1 (Start: 1.448, End: 15.18)


llama_perf_context_print:        load time =    1834.51 ms
llama_perf_context_print: prompt eval time =       0.00 ms /   173 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /   116 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =    6567.60 ms /   289 tokens
Llama.generate: 101 prefix-match hit, remaining 65 prompt tokens to eval


...

Extracted knowledge from a chunk: {'valuable_knowledge': [{'subject': 'check valve', 'predicate': 'acts as', 'object': 'ball check valve', 'timestamp': '0:00-0:08'}, {'subject': 'air pressure', 'predicate': 'causes', 'object': 'diaphragm to rise', 'timestamp': '0:09-0:16'}, {'subject': 'diaphragm', 'predicate': 'allows air to escape', 'object': 'through two ports', 'timestamp': '0:17-0:26'}, {'subject': 'air pressure', 'predicate': 'prevents air from entering', 'object': 'through the other side', 'timestamp': '0:27-0:36'}, {'subject': 'charcoal canister', 'predicate': 'contains', 'object': 'paper filter', 'timestamp': '0:37-0:41'}]}

Processing chunk 28 (Start: 534.611, End: 554.608)


llama_perf_context_print:        load time =    1834.51 ms
llama_perf_context_print: prompt eval time =       0.00 ms /    97 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /   272 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   12757.09 ms /   369 tokens
Llama.generate: 102 prefix-match hit, remaining 126 prompt tokens to eval


Extracted knowledge from a chunk: {'valuable_knowledge': [{'subject': 'activated charcoal', 'predicate': 'absorbs', 'object': 'gasoline vapors', 'timestamp': '00:00:00-00:00:10'}, {'subject': 'activated charcoal', 'predicate': 'smells like', 'object': 'gasoline', 'timestamp': '00:00:10-00:00:15'}, {'subject': 'activated charcoal', 'predicate': 'looks like', 'object': 'rat shit', 'timestamp': '00:00:15-00:00:25'}, {'subject': 'charcoal', 'predicate': 'used for', 'object': 'barbecue', 'timestamp': '00:00:35-00:00:40'}, {'subject': 'foam things', 'predicate': 'sit on', 'object': 'springs', 'timestamp': '00:00:45-00:00:55'}]}

Processing chunk 29 (Start: 554.708, End: 579.948)


llama_perf_context_print:        load time =    1834.51 ms
llama_perf_context_print: prompt eval time =       0.00 ms /   126 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /   277 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   12823.97 ms /   403 tokens


Extracted knowledge from a chunk: {'valuable_knowledge': [{'subject': 'spring with the pad', 'predicate': 'applies pressure to', 'object': 'charcoal', 'timestamp': '0:00:00-0:00:04'}, {'subject': 'two chambers', 'predicate': 'are separated by', 'object': 'a wall', 'timestamp': '0:00:10-0:00:15'}, {'subject': 'vapors', 'predicate': 'enter from', 'object': 'one side', 'timestamp': '0:00:17-0:00:21'}, {'subject': 'vapors', 'predicate': 'exit from', 'object': 'the other side', 'timestamp': '0:00:24-0:00:28'}, {'subject': 'EVAP system', 'predicate': 'works by', 'object': 'allowing maximum surface area of vapors to be absorbed through charcoal', 'timestamp': '0:00:33-0:00:40'}]}

Knowledge extraction complete.
Please check 'knowledge.json' for the extracted valuable knowledge.


## TIMESTAMPS WEG NU ALLEEN START+EIND

In [22]:
import json
import os
import pickle
import threading
from llama_cpp import Llama
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from json import JSONDecodeError

# -------------------------------
# The Chatbot and LlamaSingleton
# -------------------------------

class LlamaSingleton:
    _instance = None
    _lock = threading.Lock()

    def __new__(cls, model_path="models/Qwen2.5-7B-Instruct-Uncensored.Q4_K_M.gguf", chat_format="chatml"):
        with cls._lock:
            if cls._instance is None:
                cls._instance = super(LlamaSingleton, cls).__new__(cls)
                cls._instance.llm = Llama(model_path=model_path, chat_format=chat_format, n_ctx=2048)
            return cls._instance

class Chatbot:
    def __init__(self, 
                 messages_file='messages.json', 
                 knowledge_file='knowledge.json', 
                 faiss_index_file='faiss_index.pkl',
                 model_name='all-MiniLM-L6-v2'):
        self.messages_file = messages_file
        self.knowledge_file = knowledge_file
        self.faiss_index_file = faiss_index_file
        self.llm = LlamaSingleton().llm
        self.model = SentenceTransformer(model_name)
        self.index = None
        self.knowledge_data = []
        self.initialize_files()
        self.load_faiss_index()

    def initialize_files(self):
        for file in [self.messages_file, self.knowledge_file]:
            if not os.path.exists(file):
                with open(file, 'w') as f:
                    json.dump([], f)

    def load_json_data(self, file_path):
        with open(file_path, 'r') as f:
            return json.load(f)

    def save_json_data(self, file_path, data):
        with open(file_path, 'w') as f:
            json.dump(data, f, indent=4)

    def extract_valuable_knowledge(self, message):
        """
        Sends the chunk text to the model and asks it to return JSON with
        subject/predicate/object. No timestamps are generated by the model.
        """
        response = self.llm.create_chat_completion(
            messages=[
                {
                    "role": "system",
                    "content": (
                        "You are a knowledge extractor. Try to extract any knowledge from the user.\n"
                        "Return ONLY JSON with the following schema:\n"
                        "{\n"
                        "  \"valuable_knowledge\": [\n"
                        "    {\n"
                        "      \"subject\": \"...\",\n"
                        "      \"predicate\": \"...\",\n"
                        "      \"object\": \"...\"\n"
                        "    }\n"
                        "  ]\n"
                        "}\n"
                        "If no knowledge can be extracted, return:\n"
                        "{\"valuable_knowledge\": []}"
                    )
                },
                {"role": "user", "content": message},
            ],
            response_format={
                "type": "json",
                "schema": {
                    "type": "object",
                    "properties": {
                        "valuable_knowledge": {
                            "type": "array",
                            "items": {
                                "type": "object",
                                "properties": {
                                    "subject": {"type": "string"},
                                    "predicate": {"type": "string"},
                                    "object": {"type": "string"}
                                },
                                "required": ["subject", "predicate", "object"]
                            }
                        }
                    },
                    "required": ["valuable_knowledge"],
                },
            },
            temperature=0.5,
        )
        try:
            knowledge_data = json.loads(response['choices'][0]['message']['content'])
            print("Extracted knowledge from a chunk:", knowledge_data)
            if "valuable_knowledge" not in knowledge_data:
                knowledge_data["valuable_knowledge"] = []
            return knowledge_data["valuable_knowledge"]
        except (JSONDecodeError, KeyError):
            return []

    def save_knowledge(self, triplets):
        """
        Persists triplets to `knowledge.json` and updates FAISS index if new triplets
        are found. We do not add any timestamps here.
        """
        if not triplets:
            return
        knowledge = self.load_json_data(self.knowledge_file)
        existing_set = {(t['subject'], t['predicate'], t['object']) for t in knowledge}
        new_triplets = []
        for triplet in triplets:
            key = (triplet['subject'], triplet['predicate'], triplet['object'])
            if key not in existing_set:
                knowledge.append(triplet)
                new_triplets.append(triplet)
                existing_set.add(key)
        self.save_json_data(self.knowledge_file, knowledge)
        if new_triplets:
            self.update_faiss_index(new_triplets)

    def update_faiss_index(self, triplets):
        texts = [f"{t['subject']} {t['predicate']} {t['object']}" for t in triplets]
        embeddings = self.model.encode(texts)
        if self.index is None:
            self.index = faiss.IndexFlatL2(embeddings.shape[1])
        self.index.add(np.array(embeddings, dtype=np.float32))
        self.knowledge_data.extend(triplets)
        self.save_faiss_index()

    def save_faiss_index(self):
        with open(self.faiss_index_file, 'wb') as f:
            pickle.dump((self.index, self.knowledge_data), f)

    def load_faiss_index(self):
        if os.path.exists(self.faiss_index_file):
            with open(self.faiss_index_file, 'rb') as f:
                self.index, self.knowledge_data = pickle.load(f)
        else:
            self.index = None
            self.knowledge_data = []

# -------------------------------
# Main Processing: Load Chunks and Extract Knowledge
# -------------------------------

def main():
    # 1. Load the chunks from output_chunks.json
    with open('output_chunks.json', 'r', encoding='utf-8') as file:
        data = json.load(file)
    chunks = data.get("chunks", [])
    print(f"Total chunks loaded: {len(chunks)}")

    # 2. Extract Knowledge from Each Chunk
    chatbot = Chatbot()

    for i, chunk in enumerate(chunks, start=1):
        text = chunk.get("text", "")
        start_time = chunk.get("start")
        end_time = chunk.get("end")

        print(f"\nProcessing chunk {i} (Start: {start_time}, End: {end_time})")

        # Extract valuable knowledge from the chunk text
        extracted_knowledge = chatbot.extract_valuable_knowledge(text)

        if extracted_knowledge:
            # Attach the chunk's start/end to each extracted item
            for triplet in extracted_knowledge:
                triplet['start'] = start_time
                triplet['end'] = end_time

            # Save the extracted knowledge
            chatbot.save_knowledge(extracted_knowledge)

    print("\nKnowledge extraction complete.")
    print("Please check 'knowledge.json' for the extracted valuable knowledge.")


if __name__ == "__main__":
    main()

llama_model_load_from_file_impl: using device Metal (Apple M3 Max) - 26384 MiB free
llama_model_loader: loaded meta data with 41 key-value pairs and 339 tensors from models/Qwen2.5-7B-Instruct-Uncensored.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = qwen2
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Qwen2.5 7B Instruct Uncensored
llama_model_loader: - kv   3:                           general.finetune str              = Instruct-Uncensored
llama_model_loader: - kv   4:                           general.basename str              = Qwen2.5
llama_model_loader: - kv   5:                         general.size_label str              = 7B
llama_model_loader: - kv   6:    

Total chunks loaded: 29


load_tensors: layer   0 assigned to device CPU
load_tensors: layer   1 assigned to device CPU
load_tensors: layer   2 assigned to device CPU
load_tensors: layer   3 assigned to device CPU
load_tensors: layer   4 assigned to device CPU
load_tensors: layer   5 assigned to device CPU
load_tensors: layer   6 assigned to device CPU
load_tensors: layer   7 assigned to device CPU
load_tensors: layer   8 assigned to device CPU
load_tensors: layer   9 assigned to device CPU
load_tensors: layer  10 assigned to device CPU
load_tensors: layer  11 assigned to device CPU
load_tensors: layer  12 assigned to device CPU
load_tensors: layer  13 assigned to device CPU
load_tensors: layer  14 assigned to device CPU
load_tensors: layer  15 assigned to device CPU
load_tensors: layer  16 assigned to device CPU
load_tensors: layer  17 assigned to device CPU
load_tensors: layer  18 assigned to device CPU
load_tensors: layer  19 assigned to device CPU
load_tensors: layer  20 assigned to device CPU
load_tensors:


Processing chunk 1 (Start: 1.448, End: 15.18)


llama_perf_context_print:        load time =    1721.96 ms
llama_perf_context_print: prompt eval time =       0.00 ms /   154 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    86 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =    5247.00 ms /   240 tokens
Llama.generate: 82 prefix-match hit, remaining 65 prompt tokens to eval


...

Extracted knowledge from a chunk: {'valuable_knowledge': [{'subject': 'Check valve', 'predicate': 'acts as', 'object': 'a valve that releases air when a certain pressure is reached'}, {'subject': 'Diaphragm', 'predicate': 'functions as', 'object': 'a component that rises when air pressure builds up'}, {'subject': 'Ball check valve', 'predicate': 'is part of', 'object': 'the check valve'}, {'subject': 'Air pressure', 'predicate': 'triggers', 'object': 'the release of air'}, {'subject': 'Air', 'predicate': 'can escape', 'object': 'through two ports on the side'}, {'subject': 'Air', 'predicate': 'cannot enter', 'object': 'the canister from this side'}, {'subject': 'Paper filter', 'predicate': 'is part of', 'object': 'the charcoal canister'}]}

Processing chunk 28 (Start: 534.611, End: 554.608)


llama_perf_context_print:        load time =    1721.96 ms
llama_perf_context_print: prompt eval time =       0.00 ms /    97 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /   130 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =    6684.32 ms /   227 tokens
Llama.generate: 83 prefix-match hit, remaining 126 prompt tokens to eval


Extracted knowledge from a chunk: {'valuable_knowledge': [{'subject': 'Activated charcoal', 'predicate': 'Absorbs', 'object': 'Gasoline vapors'}, {'subject': 'Activated charcoal', 'predicate': 'Smells like', 'object': 'Gasoline'}, {'subject': 'Activated charcoal', 'predicate': 'Looks like', 'object': 'Small pieces of rat shit'}, {'subject': 'Activated charcoal', 'predicate': 'Amount', 'object': 'A lot'}]}

Processing chunk 29 (Start: 554.708, End: 579.948)


llama_perf_context_print:        load time =    1721.96 ms
llama_perf_context_print: prompt eval time =       0.00 ms /   126 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /   175 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =    9012.90 ms /   301 tokens


Extracted knowledge from a chunk: {'valuable_knowledge': [{'subject': 'spring with the pad', 'predicate': 'applies pressure', 'object': 'charcoal'}, {'subject': 'charcoal', 'predicate': 'becomes compact', 'object': 'due to pressure'}, {'subject': 'EVAP system', 'predicate': 'has two chambers', 'object': 'with a wall separating them'}, {'subject': 'vapors', 'predicate': 'enter from one side', 'object': 'and exit from the other side'}, {'subject': 'vapors', 'predicate': 'have maximum surface area', 'object': 'absorbed by charcoal'}]}

Knowledge extraction complete.
Please check 'knowledge.json' for the extracted valuable knowledge.


## TEST door te chatten

In [27]:
import json
import os
import pickle
from datetime import datetime
from llama_cpp import Llama
import threading
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from json import JSONDecodeError

class LlamaSingleton:
    _instance = None
    _lock = threading.Lock()

    def __new__(cls, model_path="models/Qwen2.5-7B-Instruct-Uncensored.Q4_K_M.gguf", chat_format="chatml"):
        with cls._lock:
            if cls._instance is None:
                cls._instance = super(LlamaSingleton, cls).__new__(cls)
                cls._instance.llm = Llama(model_path=model_path, chat_format=chat_format, n_ctx=2048)
            return cls._instance

class Chatbot:
    def __init__(self, 
                 messages_file='messages.json', 
                 knowledge_file='knowledge.json', 
                 faiss_index_file='faiss_index.pkl',
                 model_name='all-MiniLM-L6-v2'):
        self.messages_file = messages_file
        self.knowledge_file = knowledge_file
        self.faiss_index_file = faiss_index_file
        self.llm = LlamaSingleton().llm
        self.model = SentenceTransformer(model_name)
        self.index = None
        self.knowledge_data = []
        self.initialize_files()
        self.load_faiss_index()

    def initialize_files(self):
        for file in [self.messages_file, self.knowledge_file]:
            if not os.path.exists(file):
                with open(file, 'w') as f:
                    json.dump([], f)

    def load_json_data(self, file_path):
        with open(file_path, 'r') as f:
            return json.load(f)

    def save_json_data(self, file_path, data):
        with open(file_path, 'w') as f:
            json.dump(data, f, indent=4)

    def extract_valuable_knowledge(self, message):
        response = self.llm.create_chat_completion(
            messages=[
                {
                    "role": "system",
                    "content": (
                        "You are a knowledge extractor. Try to Extract any knowledge from the user.\n"
                        "Return ONLY JSON with the following schema:\n"
                        "{\n"
                        "  \"valuable_knowledge\": [\n"
                        "    {\n"
                        "      \"subject\": \"...\",\n"
                        "      \"predicate\": \"...\",\n"
                        "      \"object\": \"...\",\n"
                        "      \"timestamp\": \"...\"  # ISO8601\n"
                        "    }\n"
                        "  ]\n"
                        "}\n"
                        "If no knowledge can be extracted, return:\n"
                        "{\"valuable_knowledge\": []}"
                    )
                },
                {"role": "user", "content": message},
            ],
            response_format={
                "type": "json",
                "schema": {
                    "type": "object",
                    "properties": {
                        "valuable_knowledge": {
                            "type": "array",
                            "items": {
                                "type": "object",
                                "properties": {
                                    "subject": {"type": "string"},
                                    "predicate": {"type": "string"},
                                    "object": {"type": "string"},
                                    "timestamp": {"type": "string", "format": "date-time"}
                                },
                                "required": ["subject", "predicate", "object", "timestamp"]
                            }
                        }
                    },
                    "required": ["valuable_knowledge"],
                },
            },
            temperature=0.5,
        )
        try:
            knowledge_data = json.loads(response['choices'][0]['message']['content'])
            print(knowledge_data)
            if "valuable_knowledge" not in knowledge_data:
                knowledge_data["valuable_knowledge"] = []
            return knowledge_data["valuable_knowledge"]
        except (JSONDecodeError, KeyError):
            return []

    def save_message(self, role, content):
        messages = self.load_json_data(self.messages_file)
        message = {"role": role, "content": content, "timestamp": datetime.utcnow().isoformat()}
        messages.append(message)
        self.save_json_data(self.messages_file, messages)

    def save_knowledge(self, triplets):
        if not triplets:
            return
        knowledge = self.load_json_data(self.knowledge_file)
        existing_set = {(t['subject'], t['predicate'], t['object']) for t in knowledge}
        new_triplets = []
        for triplet in triplets:
            triplet['timestamp'] = datetime.utcnow().isoformat()
            key = (triplet['subject'], triplet['predicate'], triplet['object'])
            if key not in existing_set:
                knowledge.append(triplet)
                new_triplets.append(triplet)
                existing_set.add(key)
        self.save_json_data(self.knowledge_file, knowledge)
        if new_triplets:
            self.update_faiss_index(new_triplets)

    def update_faiss_index(self, triplets):
        texts = [f"{t['subject']} {t['predicate']} {t['object']}" for t in triplets]
        embeddings = self.model.encode(texts)
        if self.index is None:
            self.index = faiss.IndexFlatL2(embeddings.shape[1])
        self.index.add(np.array(embeddings, dtype=np.float32))
        self.knowledge_data.extend(triplets)
        self.save_faiss_index()

    def save_faiss_index(self):
        with open(self.faiss_index_file, 'wb') as f:
            pickle.dump((self.index, self.knowledge_data), f)

    def load_faiss_index(self):
        if os.path.exists(self.faiss_index_file):
            with open(self.faiss_index_file, 'rb') as f:
                self.index, self.knowledge_data = pickle.load(f)
        else:
            self.index = None
            self.knowledge_data = []

    def search_knowledge(self, query, top_k=5):
        if self.index is None or len(self.knowledge_data) == 0:
            return []
        query_embedding = self.model.encode([query])
        distances, indices = self.index.search(np.array(query_embedding, dtype=np.float32), top_k)
        results = []
        for idx in indices[0]:
            if idx == -1:
                continue
            results.append(self.knowledge_data[idx])
        return results

    def generate_response(self, conversation_history, user_message):
        knowledge_matches = None #self.search_knowledge(user_message, top_k=5)
        current_time = datetime.utcnow().isoformat()
        system_message = f"Current date and time: {current_time}\n"
        if knowledge_matches:
            system_message += "Answer based on retrieved knowledge:\n"
            for t in knowledge_matches:
                system_message += f"- {t['subject']} {t['predicate']} {t['object']} (Videotimestamps: start: {t['start']}, end: {t['end']})\n"
            
        else:
            system_message += "No direct related knowledge found. Proceeding with general reasoning.\n"
        enriched_history = [{"role": "system", "content": f"You are a helpful assistent; {system_message}"}] #+ conversation_history
        enriched_history.append({"role": "user", "content": user_message})
        print(enriched_history)
        response = self.llm.create_chat_completion(
            messages=enriched_history,
            temperature=0.7,
        )['choices'][0]['message']['content']
        return response

    def chat(self):
        print("Chatbot is ready! Type 'exit' to end the conversation.")
        while True:
            user_message = input("You: ")
            if user_message.lower().strip() in ['exit', 'quit']:
                print("Chatbot: Goodbye!")
                break
            self.save_message(role='user', content=user_message)
            conversation = self.load_json_data(self.messages_file)[-3:]
            assistant_response = self.generate_response(conversation, user_message)
            print(f"Assistant: {assistant_response}")
            #generate_speech(assistant_response)
            self.save_message(role='assistant', content=assistant_response)
            #user_knowledge_response = self.extract_valuable_knowledge(user_message)
            #print(user_knowledge_response)  
            #if user_knowledge_response:
                #self.save_knowledge(user_knowledge_response)

if __name__ == "__main__":
    chatbot = Chatbot()
    chatbot.chat()

llama_model_load_from_file_impl: using device Metal (Apple M3 Max) - 25280 MiB free
llama_model_loader: loaded meta data with 41 key-value pairs and 339 tensors from models/Qwen2.5-7B-Instruct-Uncensored.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = qwen2
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Qwen2.5 7B Instruct Uncensored
llama_model_loader: - kv   3:                           general.finetune str              = Instruct-Uncensored
llama_model_loader: - kv   4:                           general.basename str              = Qwen2.5
llama_model_loader: - kv   5:                         general.size_label str              = 7B
llama_model_loader: - kv   6:    

Chatbot is ready! Type 'exit' to end the conversation.


/var/folders/76/kx23hgz16fdcd_cjrpy0jnlm0000gn/T/ipykernel_96649/224684272.py:112: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  message = {"role": role, "content": content, "timestamp": datetime.utcnow().isoformat()}
/var/folders/76/kx23hgz16fdcd_cjrpy0jnlm0000gn/T/ipykernel_96649/224684272.py:168: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  current_time = datetime.utcnow().isoformat()


[{'role': 'system', 'content': 'You are a helpful assistent; Current date and time: 2025-02-24T13:00:41.143422\nNo direct related knowledge found. Proceeding with general reasoning.\n'}, {'role': 'user', 'content': '"question": "Which component is responsible for sensing the fuel tank\'s pressure and comparing it with atmospheric pressure?",       "options": [         "Fuel filler neck",         "Vapor pressure sensor",         "Vacuum switching valve",         "Purge line"       ],       "answer": "Vapor pressure sensor”'}]


llama_perf_context_print:        load time =    1613.15 ms
llama_perf_context_print: prompt eval time =       0.00 ms /   130 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /   379 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   17312.43 ms /   509 tokens


...

Llama.generate: 31 prefix-match hit, remaining 97 prompt tokens to eval
llama_perf_context_print:        load time =    1613.15 ms
llama_perf_context_print: prompt eval time =       0.00 ms /    97 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /   258 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   11907.38 ms /   355 tokens


Assistant: The function of the check valves in the EVAP (Evaporative Emission Control System) charcoal canister is to "Allow air in but block air out." This is necessary for maintaining the proper operation of the system.

In the EVAP system, the charcoal canister stores gasoline vapors that are emitted from the fuel tank. The system's goal is to prevent these vapors from escaping into the atmosphere. To achieve this, the system uses a series of valves and sensors.

The check valves are crucial in this process. When the engine is running, the intake manifold vacuum draws the vapor stored in the charcoal canister into the intake manifold and then into the engine for combustion. This is facilitated by check valves that allow air to enter the canister but prevent vapor from escaping back into the atmosphere.

When the engine is not running, the check valves block the air flow, keeping the vapors trapped in the canister. This prevents them from escaping into the atmosphere until the next t

Llama.generate: 31 prefix-match hit, remaining 97 prompt tokens to eval
llama_perf_context_print:        load time =    1613.15 ms
llama_perf_context_print: prompt eval time =       0.00 ms /    97 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /   296 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   13676.78 ms /   393 tokens


Assistant: The function of the check valves in the EVAP (Evaporative Emission Control System) charcoal canister is to "Allow air in but block air out". This is the correct option among the given choices.

The EVAP system is designed to capture gasoline vapors that may escape from the fuel tank and fuel lines. It does this by using a charcoal canister to store the vapors. The canister is connected to the fuel tank and the engine intake manifold through a series of valves, including check valves.

The purpose of the check valves is to allow a one-way flow of air and vapors. Specifically, they allow air to flow from the engine intake manifold into the charcoal canister to purge the stored vapors, but they prevent air from flowing directly from the atmosphere into the fuel tank. This is important because it maintains a vacuum in the fuel tank, which helps to keep the gasoline vapors contained.

The other options listed are not correct for the check valves in the EVAP system:

- "Allow air 

: 

## EVALUTION PIPELINE

In [4]:
import json
import os
import pickle
from datetime import datetime
from llama_cpp import Llama
import threading
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from json import JSONDecodeError

class LlamaSingleton:
    _instance = None
    _lock = threading.Lock()

    def __new__(cls, model_path="models/Qwen2.5-7B-Instruct.Q4_K_M.gguf", chat_format="chatml"):
        with cls._lock:
            if cls._instance is None:
                cls._instance = super(LlamaSingleton, cls).__new__(cls)
                cls._instance.llm = Llama(model_path=model_path, chat_format=chat_format, n_ctx=2048)
            return cls._instance

class Chatbot:
    def __init__(self, 
                 messages_file='messages.json', 
                 knowledge_file='knowledge.json', 
                 faiss_index_file='faiss_index.pkl',
                 model_name='all-MiniLM-L6-v2'):
        self.messages_file = messages_file
        self.knowledge_file = knowledge_file
        self.faiss_index_file = faiss_index_file
        self.llm = LlamaSingleton().llm
        self.model = SentenceTransformer(model_name)
        self.index = None
        self.knowledge_data = []
        self.initialize_files()
        self.load_faiss_index()

    def initialize_files(self):
        for file in [self.messages_file, self.knowledge_file]:
            if not os.path.exists(file):
                with open(file, 'w') as f:
                    json.dump([], f)

    def load_json_data(self, file_path):
        with open(file_path, 'r') as f:
            return json.load(f)


    def load_faiss_index(self):
        if os.path.exists(self.faiss_index_file):
            with open(self.faiss_index_file, 'rb') as f:
                self.index, self.knowledge_data = pickle.load(f)
        else:
            self.index = None
            self.knowledge_data = []

    def search_knowledge(self, query, top_k=5):
        if self.index is None or len(self.knowledge_data) == 0:
            return []
        query_embedding = self.model.encode([query])
        distances, indices = self.index.search(np.array(query_embedding, dtype=np.float32), top_k)
        results = []
        for idx in indices[0]:
            if idx == -1:
                continue
            results.append(self.knowledge_data[idx])
        return results

    def generate_response(self, user_message):
        knowledge_matches = self.search_knowledge(user_message, top_k=5)
        current_time = datetime.utcnow().isoformat()
        system_message = f"Current date and time: {current_time}\n"
        if knowledge_matches:
            system_message += "Answer based on retrieved knowledge, but only if it relates to the question:\n"
            for t in knowledge_matches:
                system_message += f"- {t['subject']} {t['predicate']} {t['object']} (Videotimestamps: start: {t['start']}, end: {t['end']})\n"
            
        else:
            system_message += "\n"
        enriched_history = [{"role": "system", "content": f"You are a helpful assistent; {system_message}"}] 
        enriched_history.append({"role": "user", "content": user_message})
        print(enriched_history)
        response = self.llm.create_chat_completion(
            messages=enriched_history,
            temperature=0.5,
        )['choices'][0]['message']['content']
        return response

    def chat(self):
        print("Chatbot is ready! Type 'exit' to end the conversation.")
        while True:
            user_message = input("You: ")
            if user_message.lower().strip() in ['exit', 'quit']:
                print("Chatbot: Goodbye!")
                break
            assistant_response = self.generate_response(user_message)
            print(f"Assistant: {assistant_response}")

if __name__ == "__main__":
    chatbot = Chatbot()
    chatbot.chat()

llama_model_load_from_file_impl: using device Metal (Apple M3 Max) - 26578 MiB free
llama_model_loader: loaded meta data with 41 key-value pairs and 339 tensors from models/Qwen2.5-7B-Instruct.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = qwen2
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Qwen2.5 7B Instruct Uncensored
llama_model_loader: - kv   3:                           general.finetune str              = Instruct-Uncensored
llama_model_loader: - kv   4:                           general.basename str              = Qwen2.5
llama_model_loader: - kv   5:                         general.size_label str              = 7B
llama_model_loader: - kv   6:               

Chatbot is ready! Type 'exit' to end the conversation.


/var/folders/76/kx23hgz16fdcd_cjrpy0jnlm0000gn/T/ipykernel_4104/140284658.py:72: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  current_time = datetime.utcnow().isoformat()


[{'role': 'system', 'content': 'You are a helpful assistent; Current date and time: 2025-02-25T18:49:12.495904\nAnswer based on retrieved knowledge:\n- valve is itself (Videotimestamps: start: 386.586, end: 387.967)\n- thing expected_to_snap soon (Videotimestamps: start: 35.679, end: 41.803)\n- spring is present in here (Videotimestamps: start: 378.841, end: 385.745)\n- check valve is present in here (Videotimestamps: start: 378.841, end: 385.745)\n- filter is present in here (Videotimestamps: start: 378.841, end: 385.745)\n'}, {'role': 'user', 'content': 'damn'}]


llama_perf_context_print:        load time =    2169.32 ms
llama_perf_context_print: prompt eval time =       0.00 ms /   232 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /   210 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   10846.34 ms /   442 tokens


Assistant: Based on the timestamps provided, it seems like the context is related to a mechanical or engineering system, possibly a hydraulic or pneumatic system. Here's a summary of the key points:

1. A valve is mentioned, and it's being described as itself (likely referring to a specific valve type or component).

2. Something expected to snap soon is mentioned, which could refer to a part like a spring or a valve component that's nearing the end of its lifespan.

3. A spring is present, which could be part of a valve mechanism or another component that operates by spring tension.

4. Check valves, filters, and other components are also present, suggesting that the system being described is complex and has multiple parts with specific functions.

Given this information, it seems like the system might be experiencing some wear and tear, with a part (possibly a spring) expected to fail soon. The presence of check valves, filters, and other components suggests that the system is design

Llama.generate: 33 prefix-match hit, remaining 222 prompt tokens to eval
llama_perf_context_print:        load time =    2169.32 ms
llama_perf_context_print: prompt eval time =       0.00 ms /   222 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    79 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =    5195.39 ms /   301 tokens


Assistant: Based on the provided timestamps from the video, it seems that the vapors enter from one side and exit from the other side of the EVAP canister. This is indicated in the video segment starting at 554.708 and ending at 579.948. Therefore, the exit point for the vapors is the opposite side from where they enter.
Chatbot: Goodbye!


## EVALUATIE SCRIPT

In [23]:
import json
import os
import pickle
import threading
import sys
from datetime import datetime
from llama_cpp import Llama
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from json import JSONDecodeError

# -------------------------------
# The Chatbot and LlamaSingleton
# -------------------------------

class LlamaSingleton:
    _instance = None
    _lock = threading.Lock()

    def __new__(cls, model_path="models/Qwen2.5-7B-Instruct.Q4_K_M.gguf", chat_format="chatml"):
        with cls._lock:
            if cls._instance is None:
                cls._instance = super(LlamaSingleton, cls).__new__(cls)
                cls._instance.llm = Llama(model_path=model_path, chat_format=chat_format, n_ctx=2048)
            return cls._instance

class Chatbot:
    def __init__(self, 
                 messages_file='messages.json', 
                 knowledge_file='knowledge.json', 
                 faiss_index_file='faiss_index.pkl',
                 model_name='all-MiniLM-L6-v2',
                 llm_model_path="models/Qwen2.5-7B-Instruct.Q4_K_M.gguf"):
        self.messages_file = messages_file
        self.knowledge_file = knowledge_file
        self.faiss_index_file = faiss_index_file
        # Instantiate LlamaSingleton with the provided model path
        self.llm = LlamaSingleton(model_path=llm_model_path).llm
        self.model = SentenceTransformer(model_name)
        self.index = None
        self.knowledge_data = []
        self.initialize_files()
        self.load_faiss_index()

    def initialize_files(self):
        for file in [self.messages_file, self.knowledge_file]:
            if not os.path.exists(file):
                with open(file, 'w') as f:
                    json.dump([], f)

    def load_json_data(self, file_path):
        with open(file_path, 'r') as f:
            return json.load(f)

    def load_faiss_index(self):
        if os.path.exists(self.faiss_index_file):
            with open(self.faiss_index_file, 'rb') as f:
                self.index, self.knowledge_data = pickle.load(f)
        else:
            self.index = None
            self.knowledge_data = []

    def search_knowledge(self, query, top_k=5):
        if self.index is None or len(self.knowledge_data) == 0:
            return []
        query_embedding = self.model.encode([query])
        distances, indices = self.index.search(np.array(query_embedding, dtype=np.float32), top_k)
        results = []
        for idx in indices[0]:
            if idx == -1:
                continue
            results.append(self.knowledge_data[idx])
        return results

    def generate_response_with_kb(self, user_message):
        knowledge_matches = self.search_knowledge(user_message, top_k=5)
        current_time = datetime.utcnow().isoformat()
        system_message = f"Current date and time: {current_time}\n"
        if knowledge_matches:
            system_message += "Choose one answer based on retrieved knowledge, if it relates to the question:\n"
            for t in knowledge_matches:
                system_message += f"- {t['subject']} {t['predicate']} {t['object']} (Videotimestamps: start: {t['start']}, end: {t['end']})\n"
        else:
            system_message += "\n"
        enriched_history = [{"role": "system", "content": f"You are a helpful assistent; {system_message}"}]
        enriched_history.append({"role": "user", "content": user_message})
        print(knowledge_matches)
        response = self.llm.create_chat_completion(
            messages=enriched_history,
            temperature=0.5,
        )['choices'][0]['message']['content']
        return response

    def generate_response_without_kb(self, user_message):
        current_time = datetime.utcnow().isoformat()
        system_message = f"Current date and time: {current_time}\n"
        enriched_history = [{"role": "system", "content": f"You are a helpful assistent. Choose one answer; {system_message}"}]
        enriched_history.append({"role": "user", "content": user_message})
        response = self.llm.create_chat_completion(
            messages=enriched_history,
            temperature=0.5,
        )['choices'][0]['message']['content']
        return response

# -------------------------------
# Evaluation: Looping Over Questions and Generating Responses
# -------------------------------

def run_evaluation(llm_model_path, questions_path):
    # Load the questions from the provided JSON file
    with open(questions_path, 'r', encoding='utf-8') as f:
        questions = json.load(f)

    # Instantiate Chatbot using the provided model path
    chatbot = Chatbot(llm_model_path=llm_model_path)

    # Loop over all questions
    for question in questions:
        # Compose a prompt that includes the question text and options.
        prompt = f"Question: {question['question']}\nOptions:\n"
        for idx, opt in enumerate(question['options']):
            prompt += f"{idx+1}. {opt}\n"

        # Generate responses using the two methods.
        response_with_kb = chatbot.generate_response_with_kb(prompt)
        response_without_kb = chatbot.generate_response_without_kb(prompt)

        # Add the responses to the question JSON object.
        question["llm_answer_with_kb"] = response_with_kb
        question["llm_answer_without_kb"] = response_without_kb

    # Derive the LLM name from the model path.
    llm_name = os.path.splitext(os.path.basename(llm_model_path))[0]
    result_filename = f"result_eval_{llm_name}.json"

    # Save the updated questions JSON to the new file.
    with open(result_filename, 'w', encoding='utf-8') as f:
        json.dump(questions, f, indent=4)

    print(f"Evaluation complete. Results saved to {result_filename}")

if __name__ == "__main__": 
    # Check command-line arguments
    model_paths = ["models/Qwen2.5-0.5B-Instruct-f16.gguf",
                   "models/Llama-3.2-1B-Instruct-f16.gguf",
                   "models/gemma-2-2b-it.F16.gguf",
                   "models/Llama-3.2-3B-Instruct-f16.gguf",
                   "models/Qwen2.5-7B-Instruct-Q4_K_M.gguf",
                   "models/Llama-3.2-11B-Vision-Instruct.Q4_K_M.gguf",
                   "models/gemma-2-9b-it-Q4_K_M.gguf",
                   "models/phi-4-14b-Q4_K_M.gguf",
                   ]
    
    for model_path in model_paths:
        questions_path = "questions.json"
        run_evaluation(model_path, questions_path)
    
    # model_path = "models/Llama-3.2-3B-Instruct-Q4_K_M.gguf"
    # questions_path = "questions.json"
    # run_evaluation(model_path, questions_path)

llama_model_load_from_file_impl: using device Metal (Apple M3 Max) - 26310 MiB free
llama_model_loader: loaded meta data with 34 key-value pairs and 290 tensors from models/Qwen2.5-0.5B-Instruct-f16.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = qwen2
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Qwen2.5 0.5B Instruct
llama_model_loader: - kv   3:                           general.finetune str              = Instruct
llama_model_loader: - kv   4:                           general.basename str              = Qwen2.5
llama_model_loader: - kv   5:                         general.size_label str              = 0.5B
llama_model_loader: - kv   6:                            genera

[{'subject': 'EVAP canister', 'predicate': 'location', 'object': 'underneath the vehicle on the right side', 'start': 1.448, 'end': 15.18}, {'subject': 'EVAP line', 'predicate': 'is held on by', 'object': 'four pieces of rust', 'start': 185.836, 'end': 194.882}, {'subject': 'canister', 'predicate': 'is_held_in_by', 'object': 'two 12mm pieces of rust', 'start': 15.2, 'end': 28.755}, {'subject': 'evap canister', 'predicate': 'has', 'object': 'two lines', 'start': 43.564, 'end': 148.419}, {'subject': 'EVAP_canister', 'predicate': 'can_be_removed', 'object': 'from above the subframe', 'start': 15.2, 'end': 28.755}]


llama_perf_context_print:        load time =     286.82 ms
llama_perf_context_print: prompt eval time =       0.00 ms /   314 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    15 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =     432.15 ms /   329 tokens
/var/folders/76/kx23hgz16fdcd_cjrpy0jnlm0000gn/T/ipykernel_4104/4187505112.py:96: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  current_time = datetime.utcnow().isoformat()
Llama.generate: 9 prefix-match hit, remaining 100 prompt tokens to eval
llama_perf_context_print:        load time =     286.82 ms
llama_perf_context_print: prompt eval time =       0.00 ms /   100 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print: 

[{'subject': 'gas tank', 'predicate': 'location', 'object': 'underneath the rear seat of the vehicle', 'start': 29.356, 'end': 35.579}, {'subject': 'fuel pump', 'predicate': 'is located', 'object': 'at the lowest part of the tank', 'start': 325.575, 'end': 365.911}, {'subject': 'EVAP canister', 'predicate': 'location', 'object': 'underneath the vehicle on the right side', 'start': 1.448, 'end': 15.18}, {'subject': 'gas tank', 'predicate': 'will be opened', 'object': "to see what's inside", 'start': 281.796, 'end': 303.949}, {'subject': 'gas tank', 'predicate': 'number of parts', 'object': 'two halves', 'start': 308.171, 'end': 325.335}]


llama_perf_context_print:        load time =     286.82 ms
llama_perf_context_print: prompt eval time =       0.00 ms /   299 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    37 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =     625.46 ms /   336 tokens
Llama.generate: 9 prefix-match hit, remaining 94 prompt tokens to eval
llama_perf_context_print:        load time =     286.82 ms
llama_perf_context_print: prompt eval time =       0.00 ms /    94 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    22 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =     368.21 ms /   116 tokens
Llama.generate: 9 prefix-match hit, remaining 292 prompt tokens to eval


...

[{'subject': 'check valve', 'predicate': 'allows fluid entry', 'object': 'into the tank', 'start': 325.575, 'end': 365.911}, {'subject': 'diaphragm', 'predicate': 'can move', 'object': 'up and down', 'start': 281.796, 'end': 303.949}, {'subject': 'Check valve', 'predicate': 'acts as', 'object': 'a valve that releases air when a certain pressure is reached', 'start': 496.587, 'end': 534.03}, {'subject': 'fuel check valve', 'predicate': 'contains', 'object': 'lid', 'start': 281.796, 'end': 303.949}, {'subject': 'maximum fuel level', 'predicate': 'is limited by', 'object': 'the height of the fuel fill check valve', 'start': 325.575, 'end': 365.911}]


llama_perf_context_print:        load time =     286.82 ms
llama_perf_context_print: prompt eval time =       0.00 ms /   320 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    56 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =     896.46 ms /   376 tokens
Llama.generate: 9 prefix-match hit, remaining 110 prompt tokens to eval
llama_perf_context_print:        load time =     286.82 ms
llama_perf_context_print: prompt eval time =       0.00 ms /   110 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    25 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =     466.62 ms /   135 tokens
Llama.generate: 9 prefix-match hit, remaining 322 prompt tokens to eval


[{'subject': 'EVAP charcoal canister', 'predicate': 'action', 'object': 'absorb gasoline vapors', 'start': 469.037, 'end': 482.005}, {'subject': 'EVAP charcoal canister', 'predicate': 'responsibility', 'object': 'hold gasoline vapors', 'start': 469.037, 'end': 482.005}, {'subject': 'check valves', 'predicate': 'function', 'object': 'control flow of gasoline vapors', 'start': 469.037, 'end': 482.005}, {'subject': 'vacuum switching valve', 'predicate': 'allows', 'object': 'air flow from gas tank to charcoal canister', 'start': 435.808, 'end': 468.877}, {'subject': 'vacuum switching valve', 'predicate': 'functions as', 'object': 'redirecting air flow between gas tank and charcoal canister', 'start': 435.808, 'end': 468.877}]


llama_perf_context_print:        load time =     286.82 ms
llama_perf_context_print: prompt eval time =       0.00 ms /   322 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    23 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =     453.61 ms /   345 tokens
Llama.generate: 9 prefix-match hit, remaining 103 prompt tokens to eval
llama_perf_context_print:        load time =     286.82 ms
llama_perf_context_print: prompt eval time =       0.00 ms /   103 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    94 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =    1065.37 ms /   197 tokens


Evaluation complete. Results saved to result_eval_phi-4-14b-Q4_K_M.json


## OVERIG: STAGE KNOWLEDGE EXTRACTION MET EXTRA VERIFICATION STEP (NOT IN FINAL BUILD)

In [2]:
# import json
# import os
# import threading
# from datetime import datetime
# from json import JSONDecodeError

# from chonkie import SentenceChunker
# from llama_cpp import Llama
# from sentence_transformers import SentenceTransformer

# # -------------------------------
# # The Chatbot and LlamaSingleton
# # -------------------------------

# class LlamaSingleton:
#     _instance = None
#     _lock = threading.Lock()

#     def __new__(cls, model_path="models/DeepSeek-R1-Distill-Qwen-1.5B-Q8_0.gguf", chat_format="chatml"):
#         with cls._lock:
#             if cls._instance is None:
#                 cls._instance = super(LlamaSingleton, cls).__new__(cls)
#                 cls._instance.llm = Llama(model_path=model_path, chat_format=chat_format, n_ctx=2048)
#             return cls._instance

# class Chatbot:
#     def __init__(self, 
#                  messages_file='messages.json', 
#                  knowledge_file='knowledge.json', 
#                  model_name='all-MiniLM-L6-v2'):
#         self.messages_file = messages_file
#         self.knowledge_file = knowledge_file
#         self.llm = LlamaSingleton().llm
#         self.model = SentenceTransformer(model_name)
#         self.knowledge_data = []
#         self.initialize_files()
#         self.load_knowledge()

#     def initialize_files(self):
#         for file in [self.messages_file, self.knowledge_file]:
#             if not os.path.exists(file):
#                 with open(file, 'w') as f:
#                     json.dump([], f)

#     def load_json_data(self, file_path):
#         with open(file_path, 'r') as f:
#             return json.load(f)

#     def save_json_data(self, file_path, data):
#         with open(file_path, 'w') as f:
#             json.dump(data, f, indent=4)

#     def load_knowledge(self):
#         self.knowledge_data = self.load_json_data(self.knowledge_file)

#     def extract_valuable_knowledge(self, text_chunk):
#         """
#         Use a two-stage chain-of-thought prompting strategy:
#           1. Extract entities, contextual metadata, and any compound/n-ary relations from the text.
#           2. Reason about the relationships among the extracted entities, self-critique the result, and refine the output.
#         """
#         # Stage 1: Extraction of entities and context
#         stage1_prompt = (
#             "You are an advanced information extractor. Given the text below, extract all relevant entities, "
#             "contextual metadata (e.g., document context, sentiment, source reliability), and any preliminary relationships "
#             "or compound relations that may exist. Return ONLY JSON following this schema:\n\n"
#             "{\n"
#             '  "entities": [{"name": "EntityName", "type": "EntityType"}],\n'
#             '  "prelim_relations": [\n'
#             "      { \"entities\": [\"EntityName1\", \"EntityName2\"], \"relation_hint\": \"...\" }\n"
#             "  ],\n"
#             '  "context": "Brief context extracted from the document",\n'
#             '  "sentiment": "positive/negative/neutral",\n'
#             '  "source_reliability": "high/medium/low"\n'
#             "}\n\n"
#             "If no relevant information is found, return empty values (e.g., empty arrays or empty strings).\n\n"
#             "Text:\n"
#             f"{text_chunk}"
#         )
#         stage1_response = self.llm.create_chat_completion(
#             messages=[
#                 {"role": "system", "content": "You are a precise and methodical information extractor."},
#                 {"role": "user", "content": stage1_prompt},
#             ],
#             temperature=0.5,
#         )

#         try:
#             stage1_data = json.loads(stage1_response['choices'][0]['message']['content'])
#         except (JSONDecodeError, KeyError):
#             stage1_data = {
#                 "entities": [],
#                 "prelim_relations": [],
#                 "context": "",
#                 "sentiment": "neutral",
#                 "source_reliability": "medium"
#             }

#         # Stage 2: Reasoning about relationships
#         stage2_prompt = (
#             "Based on the following extracted information:\n\n"
#             f"Entities: {json.dumps(stage1_data.get('entities', []))}\n"
#             f"Preliminary Relations: {json.dumps(stage1_data.get('prelim_relations', []))}\n"
#             f"Context: {stage1_data.get('context', '')}\n"
#             f"Sentiment: {stage1_data.get('sentiment', 'neutral')}\n"
#             f"Source Reliability: {stage1_data.get('source_reliability', 'medium')}\n\n"
#             "Now, reason about the relationships among these entities. Identify any compound or n-ary relations, "
#             "and provide refined relations with a confidence score (between 0 and 1). Use a chain-of-thought process "
#             "to self-critique and refine your answer. Return ONLY JSON with the following schema:\n\n"
#             "{\n"
#             '  "knowledge": [\n'
#             "      {\n"
#             '          "entities": [{"name": "EntityName", "type": "EntityType"}],\n'
#             '          "relations": [\n'
#             "              { \"entities\": [\"EntityName1\", \"EntityName2\", ...], \"predicate\": \"...\", \"confidence\": 0.95 }\n"
#             "          ],\n"
#             '          "context": "The document context",\n'
#             '          "sentiment": "positive/negative/neutral",\n'
#             '          "source_reliability": "high/medium/low",\n'
#             '          "timestamp": "ISO8601 formatted timestamp"\n'
#             "      }\n"
#             "  ]\n"
#             "}\n\n"
#             "If no refined relations can be determined, return:\n"
#             '{"knowledge": []}'
#         )

#         stage2_response = self.llm.create_chat_completion(
#             messages=[
#                 {"role": "system", "content": "You are a reflective and expert knowledge aggregator."},
#                 {"role": "user", "content": stage2_prompt},
#             ],
#             temperature=0.5,
#         )

#         try:
#             knowledge_data = json.loads(stage2_response['choices'][0]['message']['content'])
#             if "knowledge" not in knowledge_data:
#                 knowledge_data["knowledge"] = []
#         except (JSONDecodeError, KeyError):
#             knowledge_data = {"knowledge": []}

#         print("Extracted and refined knowledge from a chunk:", knowledge_data)
#         return knowledge_data["knowledge"]

#     def save_knowledge(self, new_knowledge_items):
#         """
#         Merge new knowledge items with existing ones. Use simple ontology-based reasoning to merge
#         items that refer to the same entities or concepts.
#         """
#         if not new_knowledge_items:
#             return

#         existing_knowledge = self.load_json_data(self.knowledge_file)
#         merged_knowledge = existing_knowledge.copy()

#         # A simple merging strategy: if two knowledge items share the same context and similar entities,
#         # merge their relations. (In practice, you might use a more sophisticated ontology matching algorithm.)
#         for new_item in new_knowledge_items:
#             new_item['timestamp'] = datetime.utcnow().isoformat()
#             merged = False
#             for existing_item in merged_knowledge:
#                 if existing_item.get("context") == new_item.get("context"):
#                     # Compare entity sets (by names) for overlap
#                     existing_entities = {e["name"] for e in existing_item.get("entities", [])}
#                     new_entities = {e["name"] for e in new_item.get("entities", [])}
#                     if existing_entities & new_entities:
#                         # Merge relations (avoiding duplicates based on predicate and involved entities)
#                         existing_relations = existing_item.get("relations", [])
#                         for rel in new_item.get("relations", []):
#                             if rel not in existing_relations:
#                                 existing_relations.append(rel)
#                         existing_item["relations"] = existing_relations
#                         merged = True
#                         break
#             if not merged:
#                 merged_knowledge.append(new_item)

#         self.save_json_data(self.knowledge_file, merged_knowledge)
#         print("Knowledge saved. Total items:", len(merged_knowledge))

# # -------------------------------
# # Main Processing: Chunk and Extract Knowledge
# # -------------------------------

# def main():
#     # ----------------------------
#     # Part 1: Chunk the input file
#     # ----------------------------
#     # Initialize the SentenceChunker with desired parameters
#     chunker = SentenceChunker(
#         chunk_size=512,
#         chunk_overlap=128,
#         min_sentences_per_chunk=40  # adjust as needed for text length
#     )

#     # Read the contents of input.txt
#     with open('input.txt', 'r', encoding='utf-8') as file:
#         text = file.read()

#     # Get the list of chunks from the input text
#     chunks = chunker.chunk(text)
#     print(f"Total chunks created: {len(chunks)}")

#     # ----------------------------
#     # Part 2: Extract Knowledge from Each Chunk
#     # ----------------------------
#     chatbot = Chatbot()

#     # Process each chunk
#     for i, chunk in enumerate(chunks, start=1):
#         print(f"\nProcessing chunk {i} (Token count: {chunk.token_count}, Sentences: {len(chunk.sentences)})")
#         extracted_knowledge = chatbot.extract_valuable_knowledge(chunk.text)
#         if extracted_knowledge:
#             chatbot.save_knowledge(extracted_knowledge)

#     print("\nKnowledge extraction complete.")
#     print("Please check 'knowledge.json' for the extracted valuable knowledge.")

# if __name__ == "__main__":
#     main()

llama_model_load_from_file_impl: using device Metal (Apple M3 Max) - 26602 MiB free
llama_model_loader: loaded meta data with 27 key-value pairs and 339 tensors from models/DeepSeek-R1-Distill-Qwen-1.5B-Q8_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = qwen2
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = DeepSeek R1 Distill Qwen 1.5B
llama_model_loader: - kv   3:                       general.organization str              = Deepseek Ai
llama_model_loader: - kv   4:                           general.basename str              = DeepSeek-R1-Distill-Qwen
llama_model_loader: - kv   5:                         general.size_label str              = 1.5B
llama_model_loader: - kv  

Total chunks created: 55


load_tensors: layer  23 assigned to device CPU
load_tensors: layer  24 assigned to device CPU
load_tensors: layer  25 assigned to device CPU
load_tensors: layer  26 assigned to device CPU
load_tensors: layer  27 assigned to device CPU
load_tensors: layer  28 assigned to device CPU
load_tensors: tensor 'token_embd.weight' (q8_0) (and 338 others) cannot be used with preferred buffer type CPU_AARCH64, using CPU instead
load_tensors: offloading 0 repeating layers to GPU
load_tensors: offloaded 0/29 layers to GPU
load_tensors:   CPU_Mapped model buffer size =  1801.09 MiB
llama_init_from_model: n_seq_max     = 1
llama_init_from_model: n_ctx         = 2048
llama_init_from_model: n_ctx_per_seq = 2048
llama_init_from_model: n_batch       = 512
llama_init_from_model: n_ubatch      = 512
llama_init_from_model: flash_attn    = 0
llama_init_from_model: freq_base     = 10000.0
llama_init_from_model: freq_scale    = 1
llama_init_from_model: n_ctx_per_seq (2048) < n_ctx_train (131072) -- the full cap


Processing chunk 1 (Token count: 510, Sentences: 46)


llama_perf_context_print:        load time =    1838.37 ms
llama_perf_context_print: prompt eval time =       0.00 ms /   651 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /  1396 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   26720.63 ms /  2047 tokens
Llama.generate: 12 prefix-match hit, remaining 254 prompt tokens to eval
llama_perf_context_print:        load time =    1838.37 ms
llama_perf_context_print: prompt eval time =       0.00 ms /   254 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /  1781 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   32068.14 ms /  2035 tokens
Llama.generate: 12 prefix-match hit, remaining 628 prompt tokens to eval


...

Extracted and refined knowledge from a chunk: {'knowledge': []}

Processing chunk 3 (Token count: 499, Sentences: 44)


llama_perf_context_print:        load time =    1838.37 ms
llama_perf_context_print: prompt eval time =       0.00 ms /   646 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /  1389 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   25853.42 ms /  2035 tokens
Llama.generate: 12 prefix-match hit, remaining 254 prompt tokens to eval
llama_perf_context_print:        load time =    1838.37 ms
llama_perf_context_print: prompt eval time =       0.00 ms /   254 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /  1781 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   33184.86 ms /  2035 tokens
Llama.generate: 12 prefix-match hit, remaining 647 prompt tokens to eval


Extracted and refined knowledge from a chunk: {'knowledge': []}

Processing chunk 4 (Token count: 496, Sentences: 42)


KeyboardInterrupt: 

: 